In [1202]:
import pandas as pd

In [1203]:
train_df = pd.read_csv('uid_train.txt',sep='\t',header=None)
train_df.columns = ['uid','label']

train_sms = pd.read_csv('sms_train.txt',sep='\t',header=None)
train_sms.columns = ['uid','opp_num','opp_head','opp_len','start_time','in_out']

train_voice = pd.read_csv('voice_train.txt',sep='\t',header=None)
train_voice.columns = ['uid','opp_num','opp_head','opp_len','start_time','end_time','call_type','in_out']

train_wa = pd.read_csv('wa_train.txt',sep='\t',header=None)
train_wa.columns = ['uid','wa_name','visit_cnt','visit_dura','up_flow','down_flow','wa_type','date']

C:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [1204]:
test_sms = pd.read_csv('sms_test_b.txt',sep='\t',header=None)
test_sms.columns = ['uid','opp_num','opp_head','opp_len','start_time','in_out']

test_voice = pd.read_csv('voice_test_b.txt',sep='\t',header=None)
test_voice.columns = ['uid','opp_num','opp_head','opp_len','start_time','end_time','call_type','in_out']

test_wa = pd.read_csv('wa_test_b.txt',sep='\t',header=None)
test_wa.columns = ['uid','wa_name','visit_cnt','visit_dura','up_flow','down_flow','wa_type','date']

test_df = pd.DataFrame({'uid':pd.unique(test_wa['uid'])})
test_df.to_csv('uid_test_b_v2.txt',index=None)

In [1205]:
voice = pd.concat([train_voice,test_voice],axis=0)
sms = pd.concat([train_sms,test_sms],axis=0)
wa = pd.concat([train_wa,test_wa],axis=0)

In [1206]:
train_feature = train_df
test_feature = test_df

In [1207]:
sms_feature = pd.DataFrame()
#号码总数
gp = sms.groupby('uid')['opp_num']
x = gp.apply(lambda x:x.count())
sms_feature['uid'] = x.index
sms_feature['sms_opp_num_count_all'] = x.values
#不同号码的数量
gp = sms.groupby('uid')['opp_num']
x = gp.apply(lambda x:len(set(x)))
sms_feature['sms_opp_num_count_unique'] = x.values

#头部
gp = sms.groupby('uid')['opp_head']
x = gp.apply(lambda x:x.max())
sms_feature['sms_opp_head_max'] = x.values
gp = sms.groupby('uid')['opp_head']
x = gp.apply(lambda x:x.min())
sms_feature['sms_opp_head_min'] = x.values
gp = sms.groupby('uid')['opp_head']
x = gp.apply(lambda x:x.mean())
sms_feature['sms_opp_head_mean'] = x.values

# ###
gp = sms.groupby('uid')['opp_head']
x = gp.apply(lambda x:len(set(x)))
sms_feature['sms_opp_head_unique'] = x.values
#  ###

#号码长度
gp = sms.groupby('uid')['opp_len']
x = gp.apply(lambda x:x.mean())
sms_feature['sms_opp_len_mean'] = x.values
gp = sms.groupby('uid')['opp_len']
x = gp.apply(lambda x:x.sum())
sms_feature['sms_opp_len_sum'] = x.values
gp = sms.groupby('uid')['opp_len']
x = gp.apply(lambda x:x.max())
sms_feature['sms_opp_len_max'] = x.values
# gp = sms.groupby('uid')['opp_len']
# x = gp.apply(lambda x:x.min())
# sms_feature['sms_opp_len_min'] = x.values
###
# gp = sms.groupby('uid')['opp_len']
# x = gp.apply(lambda x:len(set(x)))
# sms_feature['sms_opp_len_unique'] = x.values
# ###

#短信发生时间
gp = sms.groupby('uid')['start_time']
x = gp.apply(lambda x:x.mean())
sms_feature['sms_start_time_mean'] = x.values

#打入/打出的号码数量
gp = sms.groupby(['uid','in_out'])['opp_num']
x = gp.apply(lambda x:x.count())
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['0','1']
sms_feature['sms_opp_num_count_out'] = x['0']
sms_feature['sms_opp_num_count_in'] = x['1']
gp = sms.groupby(['uid','in_out'])['opp_num']
x = gp.apply(lambda x:len(set(x)))
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['0','1']
sms_feature['sms_opp_num_unique_out'] = x['0']
sms_feature['sms_opp_num_unique_in'] = x['1']
#打入/打出号码长度均值
gp = sms.groupby(['uid','in_out'])['opp_len']
x = gp.apply(lambda x:x.mean())
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['0','1']
sms_feature['sms_opp_len_mean_out'] = x['0']
sms_feature['sms_opp_len_mean_in'] = x['1']
# ###
# gp = sms.groupby(['uid','in_out'])['opp_len']
# x = gp.apply(lambda x:len(set(x)))
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# sms_feature['sms_opp_len_unique_out'] = x['0']
# sms_feature['sms_opp_len_unique_in'] = x['1']
# ### 
#打入/打出头部均值
gp = sms.groupby(['uid','in_out'])['opp_head']
x = gp.apply(lambda x:x.mean())
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['0','1']
sms_feature['sms_opp_head_mean_out'] = x['0']
sms_feature['sms_opp_head_mean_in'] = x['1']
gp = sms.groupby(['uid','in_out'])['opp_head']
x = gp.apply(lambda x:x.max())
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['0','1']
sms_feature['sms_opp_head_max_out'] = x['0']
sms_feature['sms_opp_head_max_in'] = x['1']
gp = sms.groupby(['uid','in_out'])['opp_head']
x = gp.apply(lambda x:x.min())
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['0','1']
sms_feature['sms_opp_head_min_out'] = x['0']
sms_feature['sms_opp_head_min_in'] = x['1']

###
# gp = sms.groupby(['uid','in_out'])['opp_head']
# x = gp.apply(lambda x:len(set(x)))
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# sms_feature['sms_opp_head_unique_out'] = x['0']
# sms_feature['sms_opp_head_unique_in'] = x['1']
# ###

#打入/打出发生时间均值
gp = sms.groupby(['uid','in_out'])['start_time']
x = gp.apply(lambda x:x.mean())
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['0','1']
sms_feature['sms_start_time_mean_out'] = x['0']
sms_feature['sms_start_time_mean_in'] = x['1']

sms['date'] = sms['start_time'] / 1000000
sms['date'] = sms['date'].astype('int')
gp = sms.groupby('uid')['date']
x = gp.apply(lambda x:len(set(x)))
sms_feature['sms_date_cnt'] = x.values

gp = sms.groupby(['uid','in_out'])['date']
x = gp.apply(lambda x:x.mean())
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['0','1']
sms_feature['sms_date_mean_out'] = x['0']
sms_feature['sms_date_mean_in'] = x['1']
gp = sms.groupby(['uid','in_out'])['date']
x = gp.apply(lambda x:len(set(x)))
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['0','1']
sms_feature['sms_date_cnt_out'] = x['0']
sms_feature['sms_date_cnt_in'] = x['1']

sms_feature['sms_callcnt_perday'] = sms_feature['sms_opp_num_count_all'] / sms_feature['sms_date_cnt']
sms_feature['sms_calluniqueout_perday'] = sms_feature['sms_opp_num_unique_out'] / sms_feature['sms_date_cnt_out']
sms_feature['sms_calluniquein_perday'] = sms_feature['sms_opp_num_unique_in'] / sms_feature['sms_date_cnt_in']
sms_feature['sms_len_perday'] = sms_feature['sms_opp_len_sum'] / sms_feature['sms_date_cnt']

# #################
sms_feature['sms_head_unique_perday'] = sms_feature['sms_opp_head_unique'] / sms_feature['sms_date_cnt']
sms_feature['sms_callunique_perday'] = sms_feature['sms_opp_num_count_unique'] / sms_feature['sms_date_cnt']
# ####################################
sms_feature['sms_calluniquecntout_perday'] = sms_feature['sms_opp_num_count_out'] / sms_feature['sms_date_cnt_out']
sms_feature['sms_calluniquecntin_perday'] = sms_feature['sms_opp_num_count_in'] / sms_feature['sms_date_cnt_in']
sms_feature['sms_calluniquemeanout_perday'] = sms_feature['sms_opp_num_count_out'] / sms_feature['sms_date_mean_out']
sms_feature['sms_calluniquemeanin_perday'] = sms_feature['sms_opp_num_count_in'] / sms_feature['sms_date_mean_in']
sms_feature['sms_calluniqueout_mean_perday'] = sms_feature['sms_opp_num_unique_out'] / sms_feature['sms_date_mean_out']
sms_feature['sms_calluniquein_mean_perday'] = sms_feature['sms_opp_num_unique_in'] / sms_feature['sms_date_mean_in']
# sms_feature['sms_head_unique_out_perday'] = sms_feature['sms_opp_head_unique_out'] / sms_feature['sms_date_cnt_out']
# sms_feature['sms_head_unique_in_perday'] = sms_feature['sms_opp_head_unique_in'] / sms_feature['sms_date_cnt_in']
# sms_feature['sms_head_unique_mean_out_perday'] = sms_feature['sms_opp_head_unique_out'] / sms_feature['sms_date_mean_out']
# sms_feature['sms_head_unique_mean_in_perday'] = sms_feature['sms_opp_head_unique_in'] / sms_feature['sms_date_mean_in']
# sms_feature['sms_len_unique_perday'] = sms_feature['sms_opp_len_unique'] / sms_feature['sms_date_cnt']
# #################

In [1208]:
# sms_date1 = pd.DataFrame()
# sms_date1 = sms[sms['date']<=9]

In [1209]:
# sms_date1_feature = pd.DataFrame()
# gp = sms_date1.groupby('uid')['date']
# x = gp.apply(lambda x:len(set(x)))
# sms_date1_feature['uid'] = x.index
# sms_date1_feature['sms_date1_visit_date_cnt'] = x.values
# #打入/打出的号码数量
# gp = sms_date1.groupby(['uid','in_out'])['opp_num']
# x = gp.apply(lambda x:x.count())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# sms_date1_feature['sms_date1_opp_num_count_out'] = x['0']
# sms_date1_feature['sms_date1_opp_num_count_in'] = x['1']
# gp = sms_date1.groupby(['uid','in_out'])['opp_num']
# x = gp.apply(lambda x:len(set(x)))
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# sms_date1_feature['sms_date1_opp_num_unique_out'] = x['0']
# sms_date1_feature['sms_date1_opp_num_unique_in'] = x['1']
# #打入/打出号码长度均值
# gp = sms_date1.groupby(['uid','in_out'])['opp_len']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# sms_date1_feature['sms_date1_opp_len_mean_out'] = x['0']
# sms_date1_feature['sms_date1_opp_len_mean_in'] = x['1']
# 
# #打入/打出头部均值
# gp = sms_date1.groupby(['uid','in_out'])['opp_head']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# sms_date1_feature['sms_date1_opp_head_mean_out'] = x['0']
# sms_date1_feature['sms_date1_opp_head_mean_in'] = x['1']
# gp = sms_date1.groupby(['uid','in_out'])['opp_head']
# x = gp.apply(lambda x:len(set(x)))
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# sms_date1_feature['sms_date1_opp_head_unique_out'] = x['0']
# sms_date1_feature['sms_date1_opp_head_unique_in'] = x['1']
# gp = sms_date1.groupby(['uid','in_out'])['opp_head']
# x = gp.apply(lambda x:x.max())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# sms_date1_feature['sms_date1_opp_head_max_out'] = x['0']
# sms_date1_feature['sms_date1_opp_head_max_in'] = x['1']
# gp = sms_date1.groupby(['uid','in_out'])['opp_head']
# x = gp.apply(lambda x:x.min())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# sms_date1_feature['sms_date1_opp_head_min_out'] = x['0']
# sms_date1_feature['sms_date1_opp_head_min_in'] = x['1']
# #打入/打出发生时间均值
# gp = sms_date1.groupby(['uid','in_out'])['start_time']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# sms_date1_feature['sms_date1_start_time_mean_out'] = x['0']
# sms_date1_feature['sms_date1_start_time_mean_in'] = x['1']
# gp = sms_date1.groupby(['uid','in_out'])['date']
# x = gp.apply(lambda x:len(set(x)))
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# sms_date1_feature['sms_date1_datecnt_out'] = x['0']
# sms_date1_feature['sms_date1_datecnt_in'] = x['1']
# 
# sms_date1_feature['sms_calluniqueout_perday'] = sms_date1_feature['sms_date1_opp_num_unique_out'] / sms_date1_feature['sms_date1_datecnt_out']
# sms_date1_feature['sms_calluniquein_perday'] = sms_date1_feature['sms_date1_opp_num_unique_in'] / sms_date1_feature['sms_date1_datecnt_in']
# sms_date1_feature['sms_calluniqueout_perday'] = sms_date1_feature['sms_date1_opp_num_count_out'] / sms_date1_feature['sms_date1_datecnt_out']
# sms_date1_feature['sms_calluniquein_perday'] = sms_date1_feature['sms_date1_opp_num_count_in'] / sms_date1_feature['sms_date1_datecnt_in']
# sms_date1_feature['sms_head_unique_out_perday'] = sms_date1_feature['sms_date1_opp_head_unique_out'] / sms_date1_feature['sms_date1_datecnt_out']
# sms_date1_feature['sms_head_unique_in_perday'] = sms_date1_feature['sms_date1_opp_head_unique_in'] / sms_date1_feature['sms_date1_datecnt_in']

In [1210]:
# train_feature = train_feature.merge(sms_date1_feature,on='uid',how='left').reset_index(drop=True)
# train_feature = train_feature.fillna(0)
# test_feature = test_feature.merge(sms_date1_feature,on='uid',how='left').reset_index(drop=True)
# test_feature = test_feature.fillna(0)

In [1211]:
# sms_date2 = pd.DataFrame()
# sms_date2 = sms[sms['date']>9]
# sms_date2 = sms_date2[sms_date2['date']<=18]

In [1212]:
# sms_date2_feature = pd.DataFrame()
# gp = sms_date2.groupby('uid')['date']
# x = gp.apply(lambda x:len(set(x)))
# sms_date2_feature['uid'] = x.index
# sms_date2_feature['sms_date2_visit_date_cnt'] = x.values
# #打入/打出的号码数量
# gp = sms_date2.groupby(['uid','in_out'])['opp_num']
# x = gp.apply(lambda x:x.count())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','2']
# sms_date2_feature['sms_date2_opp_num_count_out'] = x['0']
# sms_date2_feature['sms_date2_opp_num_count_in'] = x['2']
# gp = sms_date2.groupby(['uid','in_out'])['opp_num']
# x = gp.apply(lambda x:len(set(x)))
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','2']
# sms_date2_feature['sms_date2_opp_num_unique_out'] = x['0']
# sms_date2_feature['sms_date2_opp_num_unique_in'] = x['2']
# #打入/打出号码长度均值
# gp = sms_date2.groupby(['uid','in_out'])['opp_len']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','2']
# sms_date2_feature['sms_date2_opp_len_mean_out'] = x['0']
# sms_date2_feature['sms_date2_opp_len_mean_in'] = x['2']
# 
# #打入/打出头部均值
# gp = sms_date2.groupby(['uid','in_out'])['opp_head']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','2']
# sms_date2_feature['sms_date2_opp_head_mean_out'] = x['0']
# sms_date2_feature['sms_date2_opp_head_mean_in'] = x['2']
# gp = sms_date2.groupby(['uid','in_out'])['opp_head']
# x = gp.apply(lambda x:len(set(x)))
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','2']
# sms_date2_feature['sms_date2_opp_head_unique_out'] = x['0']
# sms_date2_feature['sms_date2_opp_head_unique_in'] = x['2']
# gp = sms_date2.groupby(['uid','in_out'])['opp_head']
# x = gp.apply(lambda x:x.max())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','2']
# sms_date2_feature['sms_date2_opp_head_max_out'] = x['0']
# sms_date2_feature['sms_date2_opp_head_max_in'] = x['2']
# gp = sms_date2.groupby(['uid','in_out'])['opp_head']
# x = gp.apply(lambda x:x.min())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','2']
# sms_date2_feature['sms_date2_opp_head_min_out'] = x['0']
# sms_date2_feature['sms_date2_opp_head_min_in'] = x['2']
# #打入/打出发生时间均值
# gp = sms_date2.groupby(['uid','in_out'])['start_time']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','2']
# sms_date2_feature['sms_date2_start_time_mean_out'] = x['0']
# sms_date2_feature['sms_date2_start_time_mean_in'] = x['2']
# gp = sms_date2.groupby(['uid','in_out'])['date']
# x = gp.apply(lambda x:len(set(x)))
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','2']
# sms_date2_feature['sms_date2_datecnt_out'] = x['0']
# sms_date2_feature['sms_date2_datecnt_in'] = x['2']
# 
# sms_date2_feature['sms_calluniqueout_perday'] = sms_date2_feature['sms_date2_opp_num_unique_out'] / sms_date2_feature['sms_date2_datecnt_out']
# sms_date2_feature['sms_calluniquein_perday'] = sms_date2_feature['sms_date2_opp_num_unique_in'] / sms_date2_feature['sms_date2_datecnt_in']
# sms_date2_feature['sms_calluniqueout_perday'] = sms_date2_feature['sms_date2_opp_num_count_out'] / sms_date2_feature['sms_date2_datecnt_out']
# sms_date2_feature['sms_calluniquein_perday'] = sms_date2_feature['sms_date2_opp_num_count_in'] / sms_date2_feature['sms_date2_datecnt_in']
# sms_date2_feature['sms_head_unique_out_perday'] = sms_date2_feature['sms_date2_opp_head_unique_out'] / sms_date2_feature['sms_date2_datecnt_out']
# sms_date2_feature['sms_head_unique_in_perday'] = sms_date2_feature['sms_date2_opp_head_unique_in'] / sms_date2_feature['sms_date2_datecnt_in']

In [1213]:
# train_feature = train_feature.merge(sms_date2_feature,on='uid',how='left').reset_index(drop=True)
# train_feature = train_feature.fillna(0)
# test_feature = test_feature.merge(sms_date2_feature,on='uid',how='left').reset_index(drop=True)
# test_feature = test_feature.fillna(0)

In [1214]:
# sms_date3 = pd.DataFrame()
# sms_date3 = sms[sms['date']>18]
# sms_date3 = sms_date3[sms_date3['date']<=27]

In [1215]:
# sms_date3_feature = pd.DataFrame()
# gp = sms_date3.groupby('uid')['date']
# x = gp.apply(lambda x:len(set(x)))
# sms_date3_feature['uid'] = x.index
# sms_date3_feature['sms_date3_visit_date_cnt'] = x.values
# #打入/打出的号码数量
# gp = sms_date3.groupby(['uid','in_out'])['opp_num']
# x = gp.apply(lambda x:x.count())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','3']
# sms_date3_feature['sms_date3_opp_num_count_out'] = x['0']
# sms_date3_feature['sms_date3_opp_num_count_in'] = x['3']
# gp = sms_date3.groupby(['uid','in_out'])['opp_num']
# x = gp.apply(lambda x:len(set(x)))
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','3']
# sms_date3_feature['sms_date3_opp_num_unique_out'] = x['0']
# sms_date3_feature['sms_date3_opp_num_unique_in'] = x['3']
# #打入/打出号码长度均值
# gp = sms_date3.groupby(['uid','in_out'])['opp_len']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','3']
# sms_date3_feature['sms_date3_opp_len_mean_out'] = x['0']
# sms_date3_feature['sms_date3_opp_len_mean_in'] = x['3']
# 
# #打入/打出头部均值
# gp = sms_date3.groupby(['uid','in_out'])['opp_head']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','3']
# sms_date3_feature['sms_date3_opp_head_mean_out'] = x['0']
# sms_date3_feature['sms_date3_opp_head_mean_in'] = x['3']
# gp = sms_date3.groupby(['uid','in_out'])['opp_head']
# x = gp.apply(lambda x:len(set(x)))
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','3']
# sms_date3_feature['sms_date3_opp_head_unique_out'] = x['0']
# sms_date3_feature['sms_date3_opp_head_unique_in'] = x['3']
# gp = sms_date3.groupby(['uid','in_out'])['opp_head']
# x = gp.apply(lambda x:x.max())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','3']
# sms_date3_feature['sms_date3_opp_head_max_out'] = x['0']
# sms_date3_feature['sms_date3_opp_head_max_in'] = x['3']
# gp = sms_date3.groupby(['uid','in_out'])['opp_head']
# x = gp.apply(lambda x:x.min())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','3']
# sms_date3_feature['sms_date3_opp_head_min_out'] = x['0']
# sms_date3_feature['sms_date3_opp_head_min_in'] = x['3']
# #打入/打出发生时间均值
# gp = sms_date3.groupby(['uid','in_out'])['start_time']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','3']
# sms_date3_feature['sms_date3_start_time_mean_out'] = x['0']
# sms_date3_feature['sms_date3_start_time_mean_in'] = x['3']
# gp = sms_date3.groupby(['uid','in_out'])['date']
# x = gp.apply(lambda x:len(set(x)))
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','3']
# sms_date3_feature['sms_date3_datecnt_out'] = x['0']
# sms_date3_feature['sms_date3_datecnt_in'] = x['3']
# 
# sms_date3_feature['sms_calluniqueout_perday'] = sms_date3_feature['sms_date3_opp_num_unique_out'] / sms_date3_feature['sms_date3_datecnt_out']
# sms_date3_feature['sms_calluniquein_perday'] = sms_date3_feature['sms_date3_opp_num_unique_in'] / sms_date3_feature['sms_date3_datecnt_in']
# sms_date3_feature['sms_calluniqueout_perday'] = sms_date3_feature['sms_date3_opp_num_count_out'] / sms_date3_feature['sms_date3_datecnt_out']
# sms_date3_feature['sms_calluniquein_perday'] = sms_date3_feature['sms_date3_opp_num_count_in'] / sms_date3_feature['sms_date3_datecnt_in']
# sms_date3_feature['sms_head_unique_out_perday'] = sms_date3_feature['sms_date3_opp_head_unique_out'] / sms_date3_feature['sms_date3_datecnt_out']
# sms_date3_feature['sms_head_unique_in_perday'] = sms_date3_feature['sms_date3_opp_head_unique_in'] / sms_date3_feature['sms_date3_datecnt_in']

In [1216]:
# train_feature = train_feature.merge(sms_date3_feature,on='uid',how='left').reset_index(drop=True)
# train_feature = train_feature.fillna(0)
# test_feature = test_feature.merge(sms_date3_feature,on='uid',how='left').reset_index(drop=True)
# test_feature = test_feature.fillna(0)

In [1217]:
# sms_date4 = pd.DataFrame()
# sms_date4 = sms[sms['date']>27]
# sms_date4 = sms_date4[sms_date4['date']<=36]

In [1218]:
# sms_date4_feature = pd.DataFrame()
# gp = sms_date4.groupby('uid')['date']
# x = gp.apply(lambda x:len(set(x)))
# sms_date4_feature['uid'] = x.index
# sms_date4_feature['sms_date4_visit_date_cnt'] = x.values
# #打入/打出的号码数量
# gp = sms_date4.groupby(['uid','in_out'])['opp_num']
# x = gp.apply(lambda x:x.count())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','4']
# sms_date4_feature['sms_date4_opp_num_count_out'] = x['0']
# sms_date4_feature['sms_date4_opp_num_count_in'] = x['4']
# gp = sms_date4.groupby(['uid','in_out'])['opp_num']
# x = gp.apply(lambda x:len(set(x)))
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','4']
# sms_date4_feature['sms_date4_opp_num_unique_out'] = x['0']
# sms_date4_feature['sms_date4_opp_num_unique_in'] = x['4']
# #打入/打出号码长度均值
# gp = sms_date4.groupby(['uid','in_out'])['opp_len']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','4']
# sms_date4_feature['sms_date4_opp_len_mean_out'] = x['0']
# sms_date4_feature['sms_date4_opp_len_mean_in'] = x['4']
# 
# #打入/打出头部均值
# gp = sms_date4.groupby(['uid','in_out'])['opp_head']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','4']
# sms_date4_feature['sms_date4_opp_head_mean_out'] = x['0']
# sms_date4_feature['sms_date4_opp_head_mean_in'] = x['4']
# gp = sms_date4.groupby(['uid','in_out'])['opp_head']
# x = gp.apply(lambda x:len(set(x)))
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','4']
# sms_date4_feature['sms_date4_opp_head_unique_out'] = x['0']
# sms_date4_feature['sms_date4_opp_head_unique_in'] = x['4']
# gp = sms_date4.groupby(['uid','in_out'])['opp_head']
# x = gp.apply(lambda x:x.max())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','4']
# sms_date4_feature['sms_date4_opp_head_max_out'] = x['0']
# sms_date4_feature['sms_date4_opp_head_max_in'] = x['4']
# gp = sms_date4.groupby(['uid','in_out'])['opp_head']
# x = gp.apply(lambda x:x.min())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','4']
# sms_date4_feature['sms_date4_opp_head_min_out'] = x['0']
# sms_date4_feature['sms_date4_opp_head_min_in'] = x['4']
# #打入/打出发生时间均值
# gp = sms_date4.groupby(['uid','in_out'])['start_time']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','4']
# sms_date4_feature['sms_date4_start_time_mean_out'] = x['0']
# sms_date4_feature['sms_date4_start_time_mean_in'] = x['4']
# gp = sms_date4.groupby(['uid','in_out'])['date']
# x = gp.apply(lambda x:len(set(x)))
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','4']
# sms_date4_feature['sms_date4_datecnt_out'] = x['0']
# sms_date4_feature['sms_date4_datecnt_in'] = x['4']
# 
# sms_date4_feature['sms_calluniqueout_perday'] = sms_date4_feature['sms_date4_opp_num_unique_out'] / sms_date4_feature['sms_date4_datecnt_out']
# sms_date4_feature['sms_calluniquein_perday'] = sms_date4_feature['sms_date4_opp_num_unique_in'] / sms_date4_feature['sms_date4_datecnt_in']
# sms_date4_feature['sms_calluniqueout_perday'] = sms_date4_feature['sms_date4_opp_num_count_out'] / sms_date4_feature['sms_date4_datecnt_out']
# sms_date4_feature['sms_calluniquein_perday'] = sms_date4_feature['sms_date4_opp_num_count_in'] / sms_date4_feature['sms_date4_datecnt_in']
# sms_date4_feature['sms_head_unique_out_perday'] = sms_date4_feature['sms_date4_opp_head_unique_out'] / sms_date4_feature['sms_date4_datecnt_out']
# sms_date4_feature['sms_head_unique_in_perday'] = sms_date4_feature['sms_date4_opp_head_unique_in'] / sms_date4_feature['sms_date4_datecnt_in']

In [1219]:
# train_feature = train_feature.merge(sms_date4_feature,on='uid',how='left').reset_index(drop=True)
# train_feature = train_feature.fillna(0)
# test_feature = test_feature.merge(sms_date4_feature,on='uid',how='left').reset_index(drop=True)
# test_feature = test_feature.fillna(0)

In [1220]:
# sms_date5 = pd.DataFrame()
# sms_date5 = sms[sms['date']>36]
# sms_date5 = sms_date4[sms_date4['date']<=45]

In [1221]:
# sms_date5_feature = pd.DataFrame()
# gp = sms_date5.groupby('uid')['date']
# x = gp.apply(lambda x:len(set(x)))
# sms_date5_feature['uid'] = x.index
# sms_date5_feature['sms_date5_visit_date_cnt'] = x.values
# #打入/打出的号码数量
# gp = sms_date5.groupby(['uid','in_out'])['opp_num']
# x = gp.apply(lambda x:x.count())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','5']
# sms_date5_feature['sms_date5_opp_num_count_out'] = x['0']
# sms_date5_feature['sms_date5_opp_num_count_in'] = x['5']
# gp = sms_date5.groupby(['uid','in_out'])['opp_num']
# x = gp.apply(lambda x:len(set(x)))
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','5']
# sms_date5_feature['sms_date5_opp_num_unique_out'] = x['0']
# sms_date5_feature['sms_date5_opp_num_unique_in'] = x['5']
# #打入/打出号码长度均值
# gp = sms_date5.groupby(['uid','in_out'])['opp_len']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','5']
# sms_date5_feature['sms_date5_opp_len_mean_out'] = x['0']
# sms_date5_feature['sms_date5_opp_len_mean_in'] = x['5']
# 
# #打入/打出头部均值
# gp = sms_date5.groupby(['uid','in_out'])['opp_head']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','5']
# sms_date5_feature['sms_date5_opp_head_mean_out'] = x['0']
# sms_date5_feature['sms_date5_opp_head_mean_in'] = x['5']
# gp = sms_date5.groupby(['uid','in_out'])['opp_head']
# x = gp.apply(lambda x:len(set(x)))
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','5']
# sms_date5_feature['sms_date5_opp_head_unique_out'] = x['0']
# sms_date5_feature['sms_date5_opp_head_unique_in'] = x['5']
# gp = sms_date5.groupby(['uid','in_out'])['opp_head']
# x = gp.apply(lambda x:x.max())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','5']
# sms_date5_feature['sms_date5_opp_head_max_out'] = x['0']
# sms_date5_feature['sms_date5_opp_head_max_in'] = x['5']
# gp = sms_date5.groupby(['uid','in_out'])['opp_head']
# x = gp.apply(lambda x:x.min())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','5']
# sms_date5_feature['sms_date5_opp_head_min_out'] = x['0']
# sms_date5_feature['sms_date5_opp_head_min_in'] = x['5']
# #打入/打出发生时间均值
# gp = sms_date5.groupby(['uid','in_out'])['start_time']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','5']
# sms_date5_feature['sms_date5_start_time_mean_out'] = x['0']
# sms_date5_feature['sms_date5_start_time_mean_in'] = x['5']
# gp = sms_date5.groupby(['uid','in_out'])['date']
# x = gp.apply(lambda x:len(set(x)))
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','5']
# sms_date5_feature['sms_date5_datecnt_out'] = x['0']
# sms_date5_feature['sms_date5_datecnt_in'] = x['5']
# 
# sms_date5_feature['sms_calluniqueout_perday'] = sms_date5_feature['sms_date5_opp_num_unique_out'] / sms_date5_feature['sms_date5_datecnt_out']
# sms_date5_feature['sms_calluniquein_perday'] = sms_date5_feature['sms_date5_opp_num_unique_in'] / sms_date5_feature['sms_date5_datecnt_in']
# sms_date5_feature['sms_calluniqueout_perday'] = sms_date5_feature['sms_date5_opp_num_count_out'] / sms_date5_feature['sms_date5_datecnt_out']
# sms_date5_feature['sms_calluniquein_perday'] = sms_date5_feature['sms_date5_opp_num_count_in'] / sms_date5_feature['sms_date5_datecnt_in']
# sms_date5_feature['sms_head_unique_out_perday'] = sms_date5_feature['sms_date5_opp_head_unique_out'] / sms_date5_feature['sms_date5_datecnt_out']
# sms_date5_feature['sms_head_unique_in_perday'] = sms_date5_feature['sms_date5_opp_head_unique_in'] / sms_date5_feature['sms_date5_datecnt_in']

In [1222]:
# train_feature = train_feature.merge(sms_date5_feature,on='uid',how='left').reset_index(drop=True)
# train_feature = train_feature.fillna(0)
# test_feature = test_feature.merge(sms_date5_feature,on='uid',how='left').reset_index(drop=True)
# test_feature = test_feature.fillna(0)

In [1223]:
# opp_len_13 = pd.DataFrame()
# opp_len_13 = sms[sms['opp_len']==13]
# gp = opp_len_13.groupby(['uid','in_out'])['opp_num']
# x = gp.apply(lambda x:x.count())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# sms_feature['sms_opp_num_13_cnt0'] = x['0']
# sms_feature['sms_opp_num_13_cnt1'] = x['1']
# gp = opp_len_13.groupby(['uid','in_out'])['opp_num']
# x = gp.apply(lambda x:len(set(x)))
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# sms_feature['sms_opp_num_13_unique0'] = x['0']
# sms_feature['sms_opp_num_13_unique1'] = x['1']
# gp = opp_len_13.groupby(['uid','in_out'])['opp_head']
# x = gp.apply(lambda x:len(set(x)))
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# sms_feature['sms_opp_head_13_unique0'] = x['0']
# sms_feature['sms_opp_head_13_unique1'] = x['1']
# gp = opp_len_13.groupby(['uid','in_out'])['opp_head']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# sms_feature['sms_opp_head_13_mean0'] = x['0']
# sms_feature['sms_opp_head_13_mean1'] = x['1']
# gp = opp_len_13.groupby(['uid','in_out'])['start_time']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# sms_feature['sms_start_time_13_mean0'] = x['0']
# sms_feature['sms_start_time_13_mean1'] = x['1']
# gp = opp_len_13.groupby(['uid','in_out'])['date']
# x = gp.apply(lambda x:len(set(x)))
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# sms_feature['sms_date_13_cnt0'] = x['0']
# sms_feature['sms_date_13_cnt1'] = x['1']

In [1224]:
# opp_len_11 = pd.DataFrame()
# opp_len_11 = sms[sms['opp_len']==11]
# gp = opp_len_11.groupby(['uid','in_out'])['opp_num']
# x = gp.apply(lambda x:x.count())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# sms_feature['sms_opp_num_11_cnt0'] = x['0']
# sms_feature['sms_opp_num_11_cnt1'] = x['1']
# gp = opp_len_11.groupby(['uid','in_out'])['opp_num']
# x = gp.apply(lambda x:len(set(x)))
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# sms_feature['sms_opp_num_11_unique0'] = x['0']
# sms_feature['sms_opp_num_11_unique1'] = x['1']
# gp = opp_len_11.groupby(['uid','in_out'])['opp_head']
# x = gp.apply(lambda x:len(set(x)))
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# sms_feature['sms_opp_head_11_unique0'] = x['0']
# sms_feature['sms_opp_head_11_unique1'] = x['1']
# gp = opp_len_11.groupby(['uid','in_out'])['opp_head']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# sms_feature['sms_opp_head_11_mean0'] = x['0']
# sms_feature['sms_opp_head_11_mean1'] = x['1']
# gp = opp_len_11.groupby(['uid','in_out'])['start_time']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# sms_feature['sms_start_time_11_mean0'] = x['0']
# sms_feature['sms_start_time_11_mean1'] = x['1']
# gp = opp_len_11.groupby(['uid','in_out'])['date']
# x = gp.apply(lambda x:len(set(x)))
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# sms_feature['sms_date_11_cnt0'] = x['0']
# sms_feature['sms_date_11_cnt1'] = x['1']

In [1225]:
# opp_len_12 = pd.DataFrame()
# opp_len_12 = sms[sms['opp_len']==12]
# gp = opp_len_12.groupby(['uid','in_out'])['opp_num']
# x = gp.apply(lambda x:x.count())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# sms_feature['sms_opp_num_12_cnt0'] = x['0']
# sms_feature['sms_opp_num_12_cnt1'] = x['1']
# gp = opp_len_12.groupby(['uid','in_out'])['opp_num']
# x = gp.apply(lambda x:len(set(x)))
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# sms_feature['sms_opp_num_12_unique0'] = x['0']
# sms_feature['sms_opp_num_12_unique1'] = x['1']
# gp = opp_len_12.groupby(['uid','in_out'])['opp_head']
# x = gp.apply(lambda x:len(set(x)))
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# sms_feature['sms_opp_head_12_unique0'] = x['0']
# sms_feature['sms_opp_head_12_unique1'] = x['1']
# gp = opp_len_12.groupby(['uid','in_out'])['opp_head']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# sms_feature['sms_opp_head_12_mean0'] = x['0']
# sms_feature['sms_opp_head_12_mean1'] = x['1']
# gp = opp_len_12.groupby(['uid','in_out'])['start_time']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# sms_feature['sms_start_time_12_mean0'] = x['0']
# sms_feature['sms_start_time_12_mean1'] = x['1']
# gp = opp_len_12.groupby(['uid','in_out'])['date']
# x = gp.apply(lambda x:len(set(x)))
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# sms_feature['sms_date_12_cnt0'] = x['0']
# sms_feature['sms_date_12_cnt1'] = x['1']

In [1226]:
train_feature = train_feature.merge(sms_feature,on='uid',how='left').reset_index(drop=True)
train_feature = train_feature.fillna(0)
test_feature = test_feature.merge(sms_feature,on='uid',how='left').reset_index(drop=True)
test_feature = test_feature.fillna(0)

In [1227]:
def head_to_int(x):
    default = '000'
    if x=='DDD':
        return default
    else:
        return x

In [1228]:
voice_feature = pd.DataFrame()
#总的通话数目
gp = voice.groupby('uid')['opp_num']
x = gp.apply(lambda x:x.count())
voice_feature['uid'] = x.index
voice_feature['voice_opp_count_all'] = x.values
#通话号码的数量
gp = voice.groupby('uid')['opp_num']
x = gp.apply(lambda x:len(set(x)))
voice_feature['voice_opp_num_unique'] = x.values

#号码长度均值
gp = voice.groupby('uid')['opp_len']
x = gp.apply(lambda x:x.mean())
voice_feature['voice_opp_len_mean'] = x.values
gp = voice.groupby('uid')['opp_len']
x = gp.apply(lambda x:x.max())
voice_feature['voice_opp_len_max'] = x.values

###
gp = voice.groupby('uid')['opp_len']
x = gp.apply(lambda x:len(set(x)))
voice_feature['voice_opp_len_unique'] = x.values
###

# 多少种头部的
gp = voice.groupby('uid')['opp_head']
x = gp.apply(lambda x:len(set(x)))
voice_feature['voice_opp_head_unique'] = x.values

#出现最大的头部
voice['opp_head'] = voice['opp_head'].apply(lambda x:head_to_int(x))
voice['opp_head'] = voice['opp_head'].astype('int')
gp = voice.groupby('uid')['opp_head']
x = gp.apply(lambda x:x.max())
voice_feature['voice_opp_head_max'] = x.values

voice['opp_head'] = voice['opp_head'].apply(lambda x:head_to_int(x))
voice['opp_head'] = voice['opp_head'].astype('int')
gp = voice.groupby('uid')['opp_head']
x = gp.apply(lambda x:x.min())
voice_feature['voice_opp_head_min'] = x.values

#出现头部的均值
voice['opp_head'] = voice['opp_head'].apply(lambda x:head_to_int(x))
voice['opp_head'] = voice['opp_head'].astype('int')
gp = voice.groupby('uid')['opp_head']
x = gp.apply(lambda x:x.mean())
voice_feature['voice_opp_head_mean'] = x.values

#通话时长均值/最大值/最小值
# voice['start_time'] = pd.to_datetime(voice['start_time'])
# voice['end_time'] = pd.to_datetime(voice['end_time'])
voice['voice_time'] = voice['end_time'] - voice['start_time']
gp = voice.groupby('uid')['voice_time']
x = gp.apply(lambda x:x.mean())
voice_feature['voice_time_mean'] = x.values
gp = voice.groupby('uid')['voice_time']
x = gp.apply(lambda x:x.sum())
voice_feature['voice_time_sum'] = x.values
gp = voice.groupby('uid')['voice_time']
x = gp.apply(lambda x:x.max())
voice_feature['voice_time_max'] = x.values
gp = voice.groupby('uid')['voice_time']
x = gp.apply(lambda x:x.min())
voice_feature['voice_time_min'] = x.values
gp = voice.groupby('uid')['voice_time']
x = gp.apply(lambda x:x.median())
voice_feature['voice_time_median'] = x.values
gp = voice.groupby('uid')['voice_time']
x = gp.apply(lambda x:x.std())
voice_feature['voice_time_std'] = x.values

#打出多少号码，打入多少号码
gp = voice.groupby(['uid','in_out'])['opp_num']
x = gp.apply(lambda x:x.count())
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['0','1']
voice_feature['voice_opp_count_out'] = x['0']
voice_feature['voice_opp_count_in'] = x['1']
#打入多少种号码，打出多少种号码
gp = voice.groupby(['uid','in_out'])['opp_num']
x = gp.apply(lambda x:len(set(x)))
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['0','1']
voice_feature['voice_opp_unique_out'] = x['0']
voice_feature['voice_opp_unique_in'] = x['1']
#打入号码的平均长度，打出号码的平均长度
gp = voice.groupby(['uid','in_out'])['opp_len']
x = gp.apply(lambda x:x.mean())
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['0','1']
voice_feature['voice_len_mean_out'] = x['0']
voice_feature['voice_len_mean_in'] = x['1']
# 
gp = voice.groupby(['uid','in_out'])['opp_len']
x = gp.apply(lambda x:x.sum())
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['0','1']
voice_feature['voice_len_sum_out'] = x['0']
voice_feature['voice_len_sum_in'] = x['1']
gp = voice.groupby(['uid','in_out'])['opp_len']
x = gp.apply(lambda x:x.max())
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['0','1']
voice_feature['voice_len_max_out'] = x['0']
voice_feature['voice_len_max_in'] = x['1']
#打入多少种头部，打出多少种头部
gp = voice.groupby(['uid','in_out'])['opp_head']
x = gp.apply(lambda x:len(set(x)))
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['0','1']
voice_feature['voice_head_unique_out'] = x['0']
voice_feature['voice_head_unique_in'] = x['1']
#打入/打出头部均值
gp = voice.groupby(['uid','in_out'])['opp_head']
x = gp.apply(lambda x:x.mean())
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['0','1']
voice_feature['voice_head_mean_out'] = x['0']
voice_feature['voice_head_mean_in'] = x['1']
#打入的平均开始时间，打出的平均开始时间
gp = voice.groupby(['uid','in_out'])['start_time']
x = gp.apply(lambda x:x.mean())
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['0','1']
voice_feature['voice_start_mean_out'] = x['0']
voice_feature['voice_start_mean_in'] = x['1']
#打入的平均时长，打出的平均时长
gp = voice.groupby(['uid','in_out'])['voice_time']
x = gp.apply(lambda x:x.mean())
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['0','1']
voice_feature['voice_time_mean_out'] = x['0']
voice_feature['voice_time_mean_in'] = x['1']
gp = voice.groupby(['uid','in_out'])['voice_time']
x = gp.apply(lambda x:x.sum())
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['0','1']
voice_feature['voice_time_sum_out'] = x['0']
voice_feature['voice_time_sum_in'] = x['1']

voice['date'] = voice['start_time']/1000000
voice['date'] = voice['date'].astype('int')
gp = voice.groupby('uid')['date']
x = gp.apply(lambda x:len(set(x)))
voice_feature['voice_date_cnt'] = x.values

gp = voice.groupby(['uid','in_out'])['date']
x = gp.apply(lambda x:x.mean())
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['0','1']
voice_feature['voice_date_mean_out'] = x['0']
voice_feature['voice_date_mean_in'] = x['1']
gp = voice.groupby(['uid','in_out'])['date']
x = gp.apply(lambda x:len(set(x)))
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['0','1']
voice_feature['voice_date_cnt_out'] = x['0']
voice_feature['voice_date_cnt_in'] = x['1']

#不同类型的通话数量
gp = voice.groupby(['uid','call_type'])['opp_num']
x = gp.apply(lambda x:x.count())
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['1','2','3','4','5']
voice_feature['voice_num_cnt_type1'] = x['1']
voice_feature['voice_num_cnt_type2'] = x['2']
voice_feature['voice_num_cnt_type3'] = x['3']
gp = voice.groupby(['uid','call_type'])['date']
x = gp.apply(lambda x:len(set(x)))
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['1','2','3','4','5']
voice_feature['voice_date_count_type1'] = x['1']
voice_feature['voice_date_count_type2'] = x['2']
voice_feature['voice_date_count_type3'] = x['3']
gp = voice.groupby(['uid','call_type'])['date']
x = gp.apply(lambda x:x.mean())
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['1','2','3','4','5']
voice_feature['voice_date_mean_type1'] = x['1']
voice_feature['voice_date_mean_type2'] = x['2']
voice_feature['voice_date_mean_type3'] = x['3']
#不同类型号码长度均值
gp = voice.groupby(['uid','call_type'])['opp_len']
x = gp.apply(lambda x:x.mean())
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['1','2','3','4','5']
voice_feature['voice_len_mean_type1'] = x['1']
# voice_feature['voice_len_mean_type2'] = x['2']
# voice_feature['voice_len_mean_type3'] = x['3']
#不同类型的头部均值
gp = voice.groupby(['uid','call_type'])['opp_head']
x = gp.apply(lambda x:x.mean())
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['1','2','3','4','5']
voice_feature['voice_head_mean_type1'] = x['1']
voice_feature['voice_head_mean_type2'] = x['2']
voice_feature['voice_head_mean_type3'] = x['3']
# 
gp = voice.groupby(['uid','call_type'])['opp_head']
x = gp.apply(lambda x:x.max())
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['1','2','3','4','5']
voice_feature['voice_head_max_type1'] = x['1']
voice_feature['voice_head_max_type2'] = x['2']
voice_feature['voice_head_max_type3'] = x['3']
#不同类型的开始时间均值
gp = voice.groupby(['uid','call_type'])['voice_time']
x = gp.apply(lambda x:x.mean())
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['1','2','3','4','5']
voice_feature['voice_time_mean_type1'] = x['1']
voice_feature['voice_time_mean_type2'] = x['2']
voice_feature['voice_time_mean_type3'] = x['3']
gp = voice.groupby(['uid','call_type'])['voice_time']
x = gp.apply(lambda x:x.sum())
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['1','2','3','4','5']
voice_feature['voice_time_sum_type1'] = x['1']
voice_feature['voice_time_sum_type2'] = x['2']
voice_feature['voice_time_sum_type3'] = x['3']


voice_feature ['voice_headout_perday'] = voice_feature['voice_head_unique_out'] / voice_feature['voice_date_cnt_out']
voice_feature['voice_headin_perday'] = voice_feature['voice_head_unique_in'] / voice_feature['voice_date_cnt_in']
voice_feature ['voice_timeoutsum_percall'] = voice_feature['voice_opp_count_out'] / voice_feature['voice_time_sum_out']
voice_feature['voice_timeinsum_percall'] = voice_feature['voice_opp_count_in'] / voice_feature['voice_time_sum_in']
voice_feature ['voice_timeoutmean_percall'] = voice_feature['voice_opp_unique_out'] / voice_feature['voice_time_mean_out']
voice_feature['voice_timeinmean_percall'] = voice_feature['voice_opp_unique_in'] / voice_feature['voice_time_mean_in']
voice_feature['voice_timesum_peruniquecall'] = voice_feature['voice_opp_num_unique'] / voice_feature['voice_time_sum']
voice_feature['voice_cnt_peruniquecall'] = voice_feature['voice_opp_num_unique'] / voice_feature['voice_date_cnt']
voice_feature ['voice_headoutunique_perday'] = voice_feature['voice_head_unique_out'] / voice_feature['voice_date_mean_out']
voice_feature['voice_headinunique_perday'] = voice_feature['voice_head_unique_in'] / voice_feature['voice_date_mean_in']
# ###############################
voice_feature['voice_head_peruniquecall'] = voice_feature['voice_opp_head_unique'] / voice_feature['voice_date_cnt']
voice_feature['voice_time1mean_percall'] = voice_feature['voice_num_cnt_type1'] / voice_feature['voice_time_mean_type1']
voice_feature['voice_time2mean_percall'] = voice_feature['voice_num_cnt_type2'] / voice_feature['voice_time_mean_type2']
voice_feature['voice_time3mean_percall'] = voice_feature['voice_num_cnt_type3'] / voice_feature['voice_time_mean_type3']
voice_feature['voice_headmean_peruniquecall'] = voice_feature['voice_opp_head_unique'] / voice_feature['voice_time_mean']
voice_feature['voice_len_peruniquecall'] = voice_feature['voice_opp_len_unique'] / voice_feature['voice_date_cnt']
voice_feature['voice_lenmean_peruniquecall'] = voice_feature['voice_opp_len_unique'] / voice_feature['voice_time_mean']


# #######################
voice_feature['voice_timemean_peruniquecall'] = voice_feature['voice_opp_num_unique'] / voice_feature['voice_time_mean']
voice_feature ['voice_timeout_percall'] = voice_feature['voice_opp_count_out'] / voice_feature['voice_time_mean_out']
voice_feature['voice_timein_percall'] = voice_feature['voice_opp_count_in'] / voice_feature['voice_time_mean_in']
voice_feature['voice_time_percall'] = voice_feature['voice_opp_count_all'] / voice_feature['voice_time_sum']
###
voice_feature['voice_timemean_percall'] = voice_feature['voice_opp_count_all'] / voice_feature['voice_time_mean']
###
voice_feature['voice_cnt_perday'] = voice_feature['voice_opp_count_all'] / voice_feature['voice_date_cnt']
# ###

voice_feature ['voice_timeoutunique_percall'] = voice_feature['voice_opp_unique_out'] / voice_feature['voice_time_sum_out']
voice_feature['voice_timeinunique_percall'] = voice_feature['voice_opp_unique_in'] / voice_feature['voice_time_sum_in']
# ###



# ###

# ###

voice_feature ['voice_cntoutsum_perday'] = voice_feature['voice_opp_count_out'] / voice_feature['voice_date_cnt_out']
voice_feature['voice_cntinsum_perday'] = voice_feature['voice_opp_count_in'] / voice_feature['voice_date_cnt_in']

# ###
voice_feature ['voice_cntoutmean_perday'] = voice_feature['voice_opp_count_out'] / voice_feature['voice_date_mean_out']
voice_feature['voice_cntinmean_perday'] = voice_feature['voice_opp_count_in'] / voice_feature['voice_date_mean_in']
# ###

voice_feature['voice_numcnt1_perday'] = voice_feature['voice_num_cnt_type1'] / voice_feature['voice_date_count_type1']
voice_feature['voice_numcnt2_perday'] = voice_feature['voice_num_cnt_type2'] / voice_feature['voice_date_count_type2']
voice_feature['voice_numcnt3_perday'] = voice_feature['voice_num_cnt_type3'] / voice_feature['voice_date_count_type3']

# ###
voice_feature['voice_numcnt1mean_perday'] = voice_feature['voice_num_cnt_type1'] / voice_feature['voice_date_mean_type1']
voice_feature['voice_numcnt2mean_perday'] = voice_feature['voice_num_cnt_type2'] / voice_feature['voice_date_mean_type2']
voice_feature['voice_numcnt3mean_perday'] = voice_feature['voice_num_cnt_type3'] / voice_feature['voice_date_mean_type3']
# ###

voice_feature['voice_time1_percall'] = voice_feature['voice_num_cnt_type1'] / voice_feature['voice_time_sum_type1']
voice_feature['voice_time2_percall'] = voice_feature['voice_num_cnt_type2'] / voice_feature['voice_time_sum_type2']
voice_feature['voice_time3_percall'] = voice_feature['voice_num_cnt_type3'] / voice_feature['voice_time_sum_type3']
voice_feature ['voice_timeoutsum_perday'] = voice_feature['voice_time_sum_out'] / voice_feature['voice_date_cnt_out']
voice_feature['voice_timeinsum_perday'] = voice_feature['voice_time_sum_in'] / voice_feature['voice_date_cnt_in']

# ###
voice_feature ['voice_timeoutmean_perday'] = voice_feature['voice_time_sum_out'] / voice_feature['voice_date_mean_out']
voice_feature['voice_timeinmean_perday'] = voice_feature['voice_time_sum_in'] / voice_feature['voice_date_mean_in']
# ###

# ###

voice_feature['voice_headsum_peruniquecall'] = voice_feature['voice_opp_head_unique'] / voice_feature['voice_time_sum']
voice_feature['voice_lensum_peruniquecall'] = voice_feature['voice_opp_len_unique'] / voice_feature['voice_time_sum']

In [1229]:
# voice_opp_len_11 = pd.DataFrame()
# voice_opp_len_11 = voice[voice['opp_len']==11]
# gp = voice_opp_len_11.groupby(['uid','in_out'])['opp_num']
# x = gp.apply(lambda x:x.count())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# voice_feature['voice_opp_len_11_opp_num_11_cnt0'] = x['0']
# voice_feature['voice_opp_len_11_opp_num_11_cnt1'] = x['1']
# gp = voice_opp_len_11.groupby(['uid','in_out'])['opp_len']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# voice_feature['voice_opp_len_11_len_mean_out'] = x['0']
# voice_feature['voice_opp_len_11_len_mean_in'] = x['1']
# gp = voice_opp_len_11.groupby(['uid','in_out'])['opp_head']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# voice_feature['voice_opp_len_11_head_mean_out'] = x['0']
# voice_feature['voice_opp_len_11_head_mean_in'] = x['1']
# gp = voice_opp_len_11.groupby(['uid','in_out'])['start_time']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# voice_feature['voice_opp_len_11_start_mean_out'] = x['0']
# voice_feature['voice_opp_len_11_start_mean_in'] = x['1']
# #打入的平均时长，打出的平均时长
# gp = voice_opp_len_11.groupby(['uid','in_out'])['voice_time']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# voice_feature['voice_opp_len_11_time_mean_out'] = x['0']
# voice_feature['voice_opp_len_11_time_mean_in'] = x['1']
# gp = voice_opp_len_11.groupby(['uid','in_out'])['date']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# voice_feature['voice_opp_len_11_date_mean_out'] = x['0']
# voice_feature['voice_opp_len_11_date_mean_in'] = x['1']
# # gp = voice_opp_len_11.groupby(['uid','call_type'])['date']
# # x = gp.apply(lambda x:x.mean())
# # x = x.unstack(fill_value=0).reset_index(drop=True)
# # x.columns = ['1','2','3','4']
# # voice_feature['voice_opp_len_11_date_mean_type1'] = x['1']
# # voice_feature['voice_opp_len_11_date_mean_type2'] = x['2']
# # voice_feature['voice_opp_len_11_date_mean_type3'] = x['3']
# # #不同类型的通话数量
# # gp = voice_opp_len_11.groupby(['uid','call_type'])['opp_num']
# # x = gp.apply(lambda x:x.count())
# # x = x.unstack(fill_value=0).reset_index(drop=True)
# # x.columns = ['1','2','3','4']
# # voice_feature['voice_opp_len_11_num_cnt_type1'] = x['1']
# # voice_feature['voice_opp_len_11_num_cnt_type2'] = x['2']
# # voice_feature['voice_opp_len_11_num_cnt_type3'] = x['3']
# # #不同类型号码长度均值
# # gp = voice_opp_len_11.groupby(['uid','call_type'])['opp_len']
# # x = gp.apply(lambda x:x.mean())
# # x = x.unstack(fill_value=0).reset_index(drop=True)
# # x.columns = ['1','2','3','4']
# # voice_feature['voice_opp_len_11_len_mean_type1'] = x['1']
# # voice_feature['voice_opp_len_11_len_mean_type2'] = x['2']
# # voice_feature['voice_opp_len_11_len_mean_type3'] = x['3']
# # #不同类型的头部均值
# # gp = voice_opp_len_11.groupby(['uid','call_type'])['opp_head']
# # x = gp.apply(lambda x:x.mean())
# # x = x.unstack(fill_value=0).reset_index(drop=True)
# # x.columns = ['1','2','3','4']
# # voice_feature['voice_opp_len_11_head_mean_type1'] = x['1']
# # voice_feature['voice_opp_len_11_head_mean_type2'] = x['2']
# # voice_feature['voice_opp_len_11_head_mean_type3'] = x['3']
# # #不同类型的开始时间均值
# # gp = voice_opp_len_11.groupby(['uid','call_type'])['voice_time']
# # x = gp.apply(lambda x:x.mean())
# # x = x.unstack(fill_value=0).reset_index(drop=True)
# # x.columns = ['1','2','3','4']
# # voice_feature['voice_opp_len_11_time_mean_type1'] = x['1']
# # voice_feature['voice_opp_len_11_time_mean_type2'] = x['2']
# # voice_feature['voice_opp_len_11_time_mean_type3'] = x['3']

In [1230]:
# voice_opp_len_13 = pd.DataFrame()
# voice_opp_len_13 = voice[voice['opp_len']==13]
# gp = voice_opp_len_13.groupby(['uid','in_out'])['opp_num']
# x = gp.apply(lambda x:x.count())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# voice_feature['voice_opp_len_13_opp_num_13_cnt0'] = x['0']
# voice_feature['voice_opp_len_13_opp_num_13_cnt1'] = x['1']
# gp = voice_opp_len_13.groupby(['uid','in_out'])['opp_len']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# voice_feature['voice_opp_len_13_len_mean_out'] = x['0']
# voice_feature['voice_opp_len_13_len_mean_in'] = x['1']
# gp = voice_opp_len_13.groupby(['uid','in_out'])['opp_head']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# voice_feature['voice_opp_len_13_head_mean_out'] = x['0']
# voice_feature['voice_opp_len_13_head_mean_in'] = x['1']
# gp = voice_opp_len_13.groupby(['uid','in_out'])['start_time']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# voice_feature['voice_opp_len_13_start_mean_out'] = x['0']
# voice_feature['voice_opp_len_13_start_mean_in'] = x['1']
# #打入的平均时长，打出的平均时长
# gp = voice_opp_len_13.groupby(['uid','in_out'])['voice_time']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# voice_feature['voice_opp_len_13_time_mean_out'] = x['0']
# voice_feature['voice_opp_len_13_time_mean_in'] = x['1']
# gp = voice_opp_len_13.groupby(['uid','in_out'])['date']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# voice_feature['voice_opp_len_13_date_mean_out'] = x['0']
# voice_feature['voice_opp_len_13_date_mean_in'] = x['1']
# 
# 
# # gp = voice_opp_len_13.groupby(['uid','call_type'])['date']
# # x = gp.apply(lambda x:x.mean())
# # x = x.unstack(fill_value=0).reset_index(drop=True)
# # x.columns = ['1','2','3','4']
# # voice_feature['voice_opp_len_13_date_mean_type1'] = x['1']
# # voice_feature['voice_opp_len_13_date_mean_type2'] = x['2']
# # voice_feature['voice_opp_len_13_date_mean_type3'] = x['3']
# # #不同类型的通话数量
# # gp = voice_opp_len_13.groupby(['uid','call_type'])['opp_num']
# # x = gp.apply(lambda x:x.count())
# # x = x.unstack(fill_value=0).reset_index(drop=True)
# # x.columns = ['1','2','3','4']
# # voice_feature['voice_opp_len_13_num_cnt_type1'] = x['1']
# # voice_feature['voice_opp_len_13_num_cnt_type2'] = x['2']
# # voice_feature['voice_opp_len_13_num_cnt_type3'] = x['3']
# # #不同类型号码长度均值
# # gp = voice_opp_len_13.groupby(['uid','call_type'])['opp_len']
# # x = gp.apply(lambda x:x.mean())
# # x = x.unstack(fill_value=0).reset_index(drop=True)
# # x.columns = ['1','2','3','4']
# # voice_feature['voice_opp_len_13_len_mean_type1'] = x['1']
# # voice_feature['voice_opp_len_13_len_mean_type2'] = x['2']
# # voice_feature['voice_opp_len_13_len_mean_type3'] = x['3']
# # #不同类型的头部均值
# # gp = voice_opp_len_13.groupby(['uid','call_type'])['opp_head']
# # x = gp.apply(lambda x:x.mean())
# # x = x.unstack(fill_value=0).reset_index(drop=True)
# # x.columns = ['1','2','3','4']
# # voice_feature['voice_opp_len_13_head_mean_type1'] = x['1']
# # voice_feature['voice_opp_len_13_head_mean_type2'] = x['2']
# # voice_feature['voice_opp_len_13_head_mean_type3'] = x['3']
# # #不同类型的开始时间均值
# # gp = voice_opp_len_13.groupby(['uid','call_type'])['voice_time']
# # x = gp.apply(lambda x:x.mean())
# # x = x.unstack(fill_value=0).reset_index(drop=True)
# # x.columns = ['1','2','3','4']
# # voice_feature['voice_opp_len_13_time_mean_type1'] = x['1']
# # voice_feature['voice_opp_len_13_time_mean_type2'] = x['2']
# # voice_feature['voice_opp_len_13_time_mean_type3'] = x['3']

In [1231]:
# voice_opp_len_12 = pd.DataFrame()
# voice_opp_len_12 = voice[voice['opp_len']==12]
# gp = voice_opp_len_12.groupby(['uid','in_out'])['opp_num']
# x = gp.apply(lambda x:x.count())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# voice_feature['voice_opp_len_12_opp_num_12_cnt0'] = x['0']
# voice_feature['voice_opp_len_12_opp_num_12_cnt1'] = x['1']
# gp = voice_opp_len_12.groupby(['uid','in_out'])['opp_len']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# voice_feature['voice_opp_len_12_len_mean_out'] = x['0']
# voice_feature['voice_opp_len_12_len_mean_in'] = x['1']
# gp = voice_opp_len_12.groupby(['uid','in_out'])['opp_head']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# voice_feature['voice_opp_len_12_head_mean_out'] = x['0']
# voice_feature['voice_opp_len_12_head_mean_in'] = x['1']
# gp = voice_opp_len_12.groupby(['uid','in_out'])['start_time']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# voice_feature['voice_opp_len_12_start_mean_out'] = x['0']
# voice_feature['voice_opp_len_12_start_mean_in'] = x['1']
# #打入的平均时长，打出的平均时长
# gp = voice_opp_len_12.groupby(['uid','in_out'])['voice_time']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# voice_feature['voice_opp_len_12_time_mean_out'] = x['0']
# voice_feature['voice_opp_len_12_time_mean_in'] = x['1']
# gp = voice_opp_len_12.groupby(['uid','in_out'])['date']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# voice_feature['voice_opp_len_12_date_mean_out'] = x['0']
# voice_feature['voice_opp_len_12_date_mean_in'] = x['1']


# gp = voice_opp_len_12.groupby(['uid','call_type'])['date']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['1','2','3']
# voice_feature['voice_opp_len_12_date_mean_type1'] = x['1']
# voice_feature['voice_opp_len_12_date_mean_type2'] = x['2']
# voice_feature['voice_opp_len_12_date_mean_type3'] = x['3']
# #不同类型的通话数量
# gp = voice_opp_len_12.groupby(['uid','call_type'])['opp_num']
# x = gp.apply(lambda x:x.count())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['1','2','3','4']
# voice_feature['voice_opp_len_12_num_cnt_type1'] = x['1']
# voice_feature['voice_opp_len_12_num_cnt_type2'] = x['2']
# voice_feature['voice_opp_len_12_num_cnt_type3'] = x['3']
# #不同类型号码长度均值
# gp = voice_opp_len_12.groupby(['uid','call_type'])['opp_len']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['1','2','3','4']
# voice_feature['voice_opp_len_12_len_mean_type1'] = x['1']
# voice_feature['voice_opp_len_12_len_mean_type2'] = x['2']
# voice_feature['voice_opp_len_12_len_mean_type3'] = x['3']
# #不同类型的头部均值
# gp = voice_opp_len_12.groupby(['uid','call_type'])['opp_head']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['1','2','3','4']
# voice_feature['voice_opp_len_12_head_mean_type1'] = x['1']
# voice_feature['voice_opp_len_12_head_mean_type2'] = x['2']
# voice_feature['voice_opp_len_12_head_mean_type3'] = x['3']
# #不同类型的开始时间均值
# gp = voice_opp_len_12.groupby(['uid','call_type'])['voice_time']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['1','2','3','4']
# voice_feature['voice_opp_len_12_time_mean_type1'] = x['1']
# voice_feature['voice_opp_len_12_time_mean_type2'] = x['2']
# voice_feature['voice_opp_len_12_time_mean_type3'] = x['3']

In [1232]:
train_feature = train_feature.merge(voice_feature,on='uid',how='left').reset_index(drop=True)
train_feature = train_feature.fillna(0)
test_feature = test_feature.merge(voice_feature,on='uid',how='left').reset_index(drop=True)
test_feature = test_feature.fillna(0)

In [1233]:
wa_feature = pd.DataFrame()
#上网总天数
gp = wa.groupby('uid')['date']
x = gp.apply(lambda x:len(set(x)))
wa_feature['uid'] = x.index
wa_feature['wa_visit_date_cnt'] = x.values
#访问次数
gp = wa.groupby('uid')['visit_cnt']
x = gp.apply(lambda x:x.sum())
wa_feature['wa_visit_cnt_sum'] = x.values
# 
gp = wa.groupby('uid')['visit_cnt']
x = gp.apply(lambda x:x.max())
wa_feature['wa_visit_cnt_max'] = x.values

gp = wa.groupby('uid')['visit_cnt']
x = gp.apply(lambda x:x.std())
wa_feature['wa_visit_cnt_std'] = x.values

gp = wa.groupby('uid')['visit_cnt']
x = gp.apply(lambda x:x.median())
wa_feature['wa_visit_cnt_median'] = x.values

gp = wa.groupby('uid')['visit_cnt']
x = gp.apply(lambda x:x.mean())
wa_feature['wa_visit_cnt_mean'] = x.values
#访问时长

gp = wa.groupby('uid')['visit_dura']
x = gp.apply(lambda x:x.sum())
wa_feature['wa_visit_dura_sum'] = x.values

gp = wa.groupby('uid')['visit_dura']
x = gp.apply(lambda x:x.max())
wa_feature['wa_visit_dura_max'] = x.values

gp = wa.groupby('uid')['visit_dura']
x = gp.apply(lambda x:x.std())
wa_feature['wa_visit_dura_std'] = x.values

gp = wa.groupby('uid')['visit_dura']
x = gp.apply(lambda x:x.median())
wa_feature['wa_visit_dura_median'] = x.values

gp = wa.groupby('uid')['visit_dura']
x = gp.apply(lambda x:x.mean())
wa_feature['wa_visit_dura_mean'] = x.values

gp = wa.groupby('uid')['down_flow']
x = gp.apply(lambda x:x.mean())
wa_feature['wa_down_flow_mean'] = x.values

gp = wa.groupby('uid')['down_flow']
x = gp.apply(lambda x:x.sum())
wa_feature['wa_down_flow_sum'] = x.values

gp = wa.groupby('uid')['down_flow']
x = gp.apply(lambda x:x.max())
wa_feature['wa_down_flow_max'] = x.values

gp = wa.groupby('uid')['down_flow']
x = gp.apply(lambda x:x.median())
wa_feature['wa_down_flow_median'] = x.values

gp = wa.groupby('uid')['down_flow']
x = gp.apply(lambda x:x.std())
wa_feature['wa_down_flow_std'] = x.values

gp = wa.groupby('uid')['up_flow']
x = gp.apply(lambda x:x.mean())
wa_feature['wa_up_flow_mean'] = x.values

gp = wa.groupby('uid')['up_flow']
x = gp.apply(lambda x:x.sum())
wa_feature['wa_up_flow_sum'] = x.values

gp = wa.groupby('uid')['up_flow']
x = gp.apply(lambda x:x.max())
wa_feature['wa_up_flow_max'] = x.values

gp = wa.groupby('uid')['up_flow']
x = gp.apply(lambda x:x.std())
wa_feature['wa_up_flow_std'] = x.values

gp = wa.groupby('uid')['up_flow']
x = gp.apply(lambda x:x.median())
wa_feature['wa_up_flow_median'] = x.values

#总流量
wa['flow'] = wa['down_flow'] + wa['up_flow']
gp = wa.groupby('uid')['flow']
x = gp.apply(lambda x:x.sum())
wa_feature['wa_flow'] =  x.values 

#APP/网站访问天数
gp = wa.groupby(['uid','wa_type'])['date']
x = gp.apply(lambda x:x.mean())
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['0','1']
wa_feature['wa_type0_date_mean'] = x['0']
wa_feature['wa_type1_date_mean'] = x['1']

gp = wa.groupby(['uid','wa_type'])['date']
x = gp.apply(lambda x:len(set(x)))
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['0','1']
wa_feature['wa_type0_date_cnt'] = x['0']
wa_feature['wa_type1_date_cnt'] = x['1']

#APP/网站访问总次数
gp = wa.groupby(['uid','wa_type'])['visit_cnt']
x = gp.apply(lambda x:x.sum())
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['0','1']
wa_feature['wa_type0_visitcnt_all'] = x['0']
wa_feature['wa_type1_visitcnt_all'] = x['1']

gp = wa.groupby(['uid','wa_type'])['visit_cnt']
x = gp.apply(lambda x:x.mean())
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['0','1']
wa_feature['wa_type0_visitcnt_mean'] = x['0']
wa_feature['wa_type1_visitcnt_mean'] = x['1']

#APP/网站上载量
gp = wa.groupby(['uid','wa_type'])['up_flow']
x = gp.apply(lambda x:x.sum())
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['0','1']
wa_feature['wa_type0_upflow_all'] = x['0']
wa_feature['wa_type1_upflow_all'] = x['1']

gp = wa.groupby(['uid','wa_type'])['up_flow']
x = gp.apply(lambda x:x.mean())
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['0','1']
wa_feature['wa_type0_upflow_mean'] = x['0']
wa_feature['wa_type1_upflow_mean'] = x['1']

#APP/网站下载量
gp = wa.groupby(['uid','wa_type'])['down_flow']
x = gp.apply(lambda x:x.sum())
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['0','1']
wa_feature['wa_type0_downflow_all'] = x['0']
wa_feature['wa_type1_downflow_all'] = x['1']
gp = wa.groupby(['uid','wa_type'])['down_flow']
x = gp.apply(lambda x:x.mean())
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['0','1']
wa_feature['wa_type0_downflow_mean'] = x['0']
wa_feature['wa_type1_downflow_mean'] = x['1']

gp = wa.groupby(['uid','wa_type'])['flow']
x = gp.apply(lambda x:x.mean())
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['0','1']
wa_feature['wa_type0_flow_mean'] = x['0']
wa_feature['wa_type1_flow_mean'] = x['1']

#APP/网站访问平均时长
gp = wa.groupby(['uid','wa_type'])['visit_dura']
x = gp.apply(lambda x:x.mean())
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['0','1']
wa_feature['wa_type0_dura_mean'] = x['0']
wa_feature['wa_type1_dura_mean'] = x['1']

gp=wa.groupby('uid')['wa_name']
x=gp.apply(lambda x:len(set(x)))
wa_feature['wa_name_count_uinque']=x.values

#APP/网站分别访问了多少种
gp = wa.groupby(['uid','wa_type'])['wa_name']
x = gp.apply(lambda x:len(set(x)))
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['0','1']
wa_feature['wa_type0_name_unique'] = x['0']
wa_feature['wa_type1_name_unique'] = x['1']


wa_feature['wa_namecnt_perday'] = wa_feature['wa_name_count_uinque'] / wa_feature['wa_visit_date_cnt']
wa_feature['wa_dura_sum_percnt'] =  wa_feature['wa_visit_dura_sum'] / wa_feature['wa_visit_cnt_sum']
wa_feature['wa_type0_namecnt_perday'] = wa_feature['wa_type0_name_unique'] / wa_feature['wa_type0_date_cnt']
wa_feature['wa_type1_namecnt_perday'] = wa_feature['wa_type1_name_unique'] / wa_feature['wa_type1_date_cnt']
wa_feature['wa_type0_namedura_perday'] = wa_feature['wa_type0_name_unique'] / wa_feature['wa_type0_dura_mean']
wa_feature['wa_type1_namedura_perday'] = wa_feature['wa_type1_name_unique'] / wa_feature['wa_type1_dura_mean']
wa_feature['wa_type0_namevicnt_perday'] = wa_feature['wa_type0_name_unique'] / wa_feature['wa_type0_visitcnt_mean']
wa_feature['wa_type1_namevicnt_perday'] = wa_feature['wa_type1_name_unique'] / wa_feature['wa_type1_visitcnt_mean']
wa_feature['wa_type0_namevicntall_perday'] = wa_feature['wa_type0_name_unique'] / wa_feature['wa_type0_visitcnt_all']
wa_feature['wa_type1_namevicntall_perday'] = wa_feature['wa_type1_name_unique'] / wa_feature['wa_type1_visitcnt_all']
wa_feature['wa_type0_dura_perday'] = wa_feature['wa_type0_dura_mean'] / wa_feature['wa_type0_date_cnt']
wa_feature['wa_type1_dura_perday'] = wa_feature['wa_type1_dura_mean'] / wa_feature['wa_type1_date_cnt']
wa_feature['wa_type0_up_persec'] = wa_feature['wa_type0_upflow_all'] / wa_feature['wa_type0_dura_mean']
wa_feature['wa_type1_up_persec'] = wa_feature['wa_type1_upflow_all'] / wa_feature['wa_type1_dura_mean']
wa_feature['wa_type0_down_percnt'] = wa_feature['wa_type0_downflow_all'] / wa_feature['wa_type0_visitcnt_mean']
wa_feature['wa_type1_down_percnt'] = wa_feature['wa_type1_downflow_all'] / wa_feature['wa_type1_visitcnt_mean']

# #######################################
wa_feature['wa_visitdura_perday'] = wa_feature['wa_visit_dura_sum'] / wa_feature['wa_visit_date_cnt']
wa_feature['wa_visitcnt_perday'] = wa_feature['wa_visit_cnt_sum'] / wa_feature['wa_visit_date_cnt']
wa_feature['wa_namedura_perday'] = wa_feature['wa_name_count_uinque'] / wa_feature['wa_visit_dura_sum']
wa_feature['wa_namevicnt_perday'] = wa_feature['wa_name_count_uinque'] / wa_feature['wa_visit_cnt_sum']
wa_feature['wa_type0_visitcnt_perday'] = wa_feature['wa_type0_visitcnt_all'] / wa_feature['wa_type0_date_cnt']
wa_feature['wa_type1_visitcnt_perday'] = wa_feature['wa_type1_visitcnt_all'] / wa_feature['wa_type1_date_cnt']
wa_feature['wa_flow_sum_percnt'] =  wa_feature['wa_flow'] /wa_feature['wa_visit_cnt_sum']
wa_feature['wa_flow_sum_persecond'] =  wa_feature['wa_flow'] / wa_feature['wa_visit_dura_sum']
wa_feature['wa_flow_sum_perday'] = wa_feature['wa_flow'] / wa_feature['wa_visit_date_cnt']
wa_feature['wa_upflow_perday'] = wa_feature['wa_up_flow_sum'] / wa_feature['wa_visit_date_cnt']
wa_feature['wa_downflow_perday'] = wa_feature['wa_down_flow_sum'] / wa_feature['wa_visit_date_cnt']
wa_feature['wa_upflowcnt_perday'] = wa_feature['wa_up_flow_sum'] / wa_feature['wa_visit_cnt_mean']
wa_feature['wa_downflowcnt_perday'] = wa_feature['wa_down_flow_sum'] / wa_feature['wa_visit_cnt_mean']
wa_feature['wa_upflowdura_persec'] = wa_feature['wa_up_flow_sum'] / wa_feature['wa_visit_dura_mean']
wa_feature['wa_downflowdura_persec'] = wa_feature['wa_down_flow_sum'] / wa_feature['wa_visit_dura_mean']
wa_feature['wa_type0_up_perday'] = wa_feature['wa_type0_upflow_all'] / wa_feature['wa_type0_date_cnt']
wa_feature['wa_type1_up_perday'] = wa_feature['wa_type1_upflow_all'] / wa_feature['wa_type1_date_cnt']
wa_feature['wa_type0_down_perday'] = wa_feature['wa_type0_downflow_all'] / wa_feature['wa_type0_date_cnt']
wa_feature['wa_type1_down_perday'] = wa_feature['wa_type1_downflow_all'] / wa_feature['wa_type1_date_cnt']


In [1234]:
train_feature = train_feature.merge(wa_feature,on='uid',how='left').reset_index(drop=True)
train_feature = train_feature.fillna(0)
test_feature = test_feature.merge(wa_feature,on='uid',how='left').reset_index(drop=True)
test_feature = test_feature.fillna(0)

In [1235]:
# date1 = pd.DataFrame()
# date1 = wa[wa['date']<=9]

In [1236]:
# date1_feature = pd.DataFrame()
# #上网总天数
# gp = date1.groupby('uid')['date']
# x = gp.apply(lambda x:len(set(x)))
# date1_feature['uid'] = x.index
# date1_feature['date1_visit_date_cnt'] = x.values
# 
# gp = date1.groupby('uid')['visit_cnt']
# x = gp.apply(lambda x:x.mean())
# date1_feature['date1_visit_cnt_mean'] = x.values
# #访问时长
# # 
# gp = date1.groupby('uid')['visit_dura']
# x = gp.apply(lambda x:x.sum())
# date1_feature['date1_visit_dura_sum'] = x.values
# 
# gp = date1.groupby('uid')['visit_dura']
# x = gp.apply(lambda x:x.mean())
# date1_feature['date1_visit_dura_mean'] = x.values
# 
# gp = date1.groupby('uid')['down_flow']
# x = gp.apply(lambda x:x.mean())
# date1_feature['date1_down_flow_mean'] = x.values
# 
# gp = date1.groupby('uid')['up_flow']
# x = gp.apply(lambda x:x.mean())
# date1_feature['date1_up_flow_mean'] = x.values
# 
# #总流量
# date1['flow'] = date1['down_flow'] + date1['up_flow']
# gp = date1.groupby('uid')['flow']
# x = gp.apply(lambda x:x.sum())
# date1_feature['date1_flow_sum'] = x.values
# 
# gp = date1.groupby('uid')['date']
# y = gp.apply(lambda x:len(set(x)))
# date1_feature['date1_flow_perday_mean'] = date1_feature['date1_flow_sum'] / y.values
# 
# gp = date1.groupby('uid')['visit_cnt']
# x = gp.apply(lambda x:x.sum())
# date1_feature['date1_flow_sum_percnt'] = date1_feature['date1_flow_sum'] / x.values 
# 
# gp = date1.groupby('uid')['visit_cnt']
# x = gp.apply(lambda x:x.sum())
# date1_feature['date1_dura_sum_percnt'] = date1_feature['date1_visit_dura_sum'] / x.values 
# 
# #APP/网站访问天数
# gp = date1.groupby(['uid','wa_type'])['date']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# date1_feature['date1_type0_date_mean'] = x['0']
# date1_feature['date1_type1_date_mean'] = x['1']
# 
# gp = date1.groupby(['uid','wa_type'])['date']
# x = gp.apply(lambda x:x.count())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# date1_feature['date1_type0_date_cnt'] = x['0']
# date1_feature['date1_type1_date_cnt'] = x['1']
# 
# #APP/网站访问总次数
# 
# gp = date1.groupby(['uid','wa_type'])['visit_cnt']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# date1_feature['date1_type0_visitcnt_mean'] = x['0']
# date1_feature['date1_type1_visitcnt_mean'] = x['1']
# 
# #APP/网站上载量
# gp = date1.groupby(['uid','wa_type'])['up_flow']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# date1_feature['date1_type0_upflow_mean'] = x['0']
# date1_feature['date1_type1_upflow_mean'] = x['1']
# #APP/网站下载量
# gp = date1.groupby(['uid','wa_type'])['down_flow']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# date1_feature['date1_type0_downflow_mean'] = x['0']
# date1_feature['date1_type1_downflow_mean'] = x['1']
# gp = date1.groupby(['uid','wa_type'])['flow']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# date1_feature['date1_type0_flow_mean'] = x['0']
# date1_feature['date1_type1_flow_mean'] = x['1']
# #APP/网站访问平均时长
# gp = date1.groupby(['uid','wa_type'])['visit_dura']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# date1_feature['date1_type0_dura_mean'] = x['0']
# date1_feature['date1_type1_dura_mean'] = x['1']
# 
# gp=date1.groupby('uid')['wa_name']
# x=gp.apply(lambda x:len(set(x)))
# date1_feature['date1_name_count_uinque']=x.values
# #APP/网站分别访问了多少种
# gp = date1.groupby(['uid','wa_type'])['wa_name']
# x = gp.apply(lambda x:len(set(x)))
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','1']
# date1_feature['date1_type0_name_unique'] = x['0']
# date1_feature['date1_type1_name_unique'] = x['1']

In [1237]:
# train_feature = train_feature.merge(date1_feature,on='uid',how='left').reset_index(drop=True)
# train_feature = train_feature.fillna(0)
# test_feature = test_feature.merge(date1_feature,on='uid',how='left').reset_index(drop=True)
# test_feature = test_feature.fillna(0)

In [1238]:
# date2 = pd.DataFrame()
# date2 = wa[wa['date']>9]
# date2 = date2[date2['date']<=18]

In [1239]:
# date2_feature = pd.DataFrame()
# #上网总天数
# gp = date2.groupby('uid')['date']
# x = gp.apply(lambda x:len(set(x)))
# date2_feature['uid'] = x.index
# date2_feature['date2_visit_date_cnt'] = x.values
# 
# gp = date2.groupby('uid')['visit_cnt']
# x = gp.apply(lambda x:x.mean())
# date2_feature['date2_visit_cnt_mean'] = x.values
# #访问时长
# # 
# gp = date2.groupby('uid')['visit_dura']
# x = gp.apply(lambda x:x.sum())
# date2_feature['date2_visit_dura_sum'] = x.values
# 
# gp = date2.groupby('uid')['visit_dura']
# x = gp.apply(lambda x:x.mean())
# date2_feature['date2_visit_dura_mean'] = x.values
# 
# gp = date2.groupby('uid')['down_flow']
# x = gp.apply(lambda x:x.mean())
# date2_feature['date2_down_flow_mean'] = x.values
# 
# gp = date2.groupby('uid')['up_flow']
# x = gp.apply(lambda x:x.mean())
# date2_feature['date2_up_flow_mean'] = x.values
# 
# #总流量
# date2['flow'] = date2['down_flow'] + date2['up_flow']
# gp = date2.groupby('uid')['flow']
# x = gp.apply(lambda x:x.sum())
# date2_feature['date2_flow_sum'] = x.values
# 
# gp = date2.groupby('uid')['date']
# y = gp.apply(lambda x:len(set(x)))
# date2_feature['date2_flow_perday_mean'] = date2_feature['date2_flow_sum'] / y.values
# 
# gp = date2.groupby('uid')['visit_cnt']
# x = gp.apply(lambda x:x.sum())
# date2_feature['date2_flow_sum_percnt'] = date2_feature['date2_flow_sum'] / x.values 
# 
# gp = date2.groupby('uid')['visit_cnt']
# x = gp.apply(lambda x:x.sum())
# date2_feature['date2_dura_sum_percnt'] = date2_feature['date2_visit_dura_sum'] / x.values 
# 
# #APP/网站访问天数
# gp = date2.groupby(['uid','wa_type'])['date']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','2']
# date2_feature['date2_type0_date_mean'] = x['0']
# date2_feature['date2_type2_date_mean'] = x['2']
# 
# gp = date2.groupby(['uid','wa_type'])['date']
# x = gp.apply(lambda x:x.count())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','2']
# date2_feature['date2_type0_date_cnt'] = x['0']
# date2_feature['date2_type2_date_cnt'] = x['2']
# 
# #APP/网站访问总次数
# 
# gp = date2.groupby(['uid','wa_type'])['visit_cnt']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','2']
# date2_feature['date2_type0_visitcnt_mean'] = x['0']
# date2_feature['date2_type2_visitcnt_mean'] = x['2']
# 
# #APP/网站上载量
# gp = date2.groupby(['uid','wa_type'])['up_flow']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','2']
# date2_feature['date2_type0_upflow_mean'] = x['0']
# date2_feature['date2_type2_upflow_mean'] = x['2']
# #APP/网站下载量
# gp = date2.groupby(['uid','wa_type'])['down_flow']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','2']
# date2_feature['date2_type0_downflow_mean'] = x['0']
# date2_feature['date2_type2_downflow_mean'] = x['2']
# gp = date2.groupby(['uid','wa_type'])['flow']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','2']
# date2_feature['date2_type0_flow_mean'] = x['0']
# date2_feature['date2_type2_flow_mean'] = x['2']
# #APP/网站访问平均时长
# gp = date2.groupby(['uid','wa_type'])['visit_dura']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','2']
# date2_feature['date2_type0_dura_mean'] = x['0']
# date2_feature['date2_type2_dura_mean'] = x['2']
# 
# gp=date2.groupby('uid')['wa_name']
# x=gp.apply(lambda x:len(set(x)))
# date2_feature['date2_name_count_uinque']=x.values
# #APP/网站分别访问了多少种
# gp = date2.groupby(['uid','wa_type'])['wa_name']
# x = gp.apply(lambda x:len(set(x)))
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','2']
# date2_feature['date2_type0_name_unique'] = x['0']
# date2_feature['date2_type2_name_unique'] = x['2']

In [1240]:
# train_feature = train_feature.merge(date2_feature,on='uid',how='left').reset_index(drop=True)
# train_feature = train_feature.fillna(0)
# test_feature = test_feature.merge(date2_feature,on='uid',how='left').reset_index(drop=True)
# test_feature = test_feature.fillna(0)

In [1241]:
# date3 = pd.DataFrame()
# date3 = wa[wa['date']>18]
# date3 = date3[date3['date']<=27]

In [1242]:
# date3_feature = pd.DataFrame()
# #上网总天数
# gp = date3.groupby('uid')['date']
# x = gp.apply(lambda x:len(set(x)))
# date3_feature['uid'] = x.index
# date3_feature['date3_visit_date_cnt'] = x.values
# 
# gp = date3.groupby('uid')['visit_cnt']
# x = gp.apply(lambda x:x.mean())
# date3_feature['date3_visit_cnt_mean'] = x.values
# #访问时长
# # 
# gp = date3.groupby('uid')['visit_dura']
# x = gp.apply(lambda x:x.sum())
# date3_feature['date3_visit_dura_sum'] = x.values
# 
# gp = date3.groupby('uid')['visit_dura']
# x = gp.apply(lambda x:x.mean())
# date3_feature['date3_visit_dura_mean'] = x.values
# 
# gp = date3.groupby('uid')['down_flow']
# x = gp.apply(lambda x:x.mean())
# date3_feature['date3_down_flow_mean'] = x.values
# 
# gp = date3.groupby('uid')['up_flow']
# x = gp.apply(lambda x:x.mean())
# date3_feature['date3_up_flow_mean'] = x.values
# 
# #总流量
# date3['flow'] = date3['down_flow'] + date3['up_flow']
# gp = date3.groupby('uid')['flow']
# x = gp.apply(lambda x:x.sum())
# date3_feature['date3_flow_sum'] = x.values
# 
# gp = date3.groupby('uid')['date']
# y = gp.apply(lambda x:len(set(x)))
# date3_feature['date3_flow_perday_mean'] = date3_feature['date3_flow_sum'] / y.values
# 
# gp = date3.groupby('uid')['visit_cnt']
# x = gp.apply(lambda x:x.sum())
# date3_feature['date3_flow_sum_percnt'] = date3_feature['date3_flow_sum'] / x.values 
# 
# gp = date3.groupby('uid')['visit_cnt']
# x = gp.apply(lambda x:x.sum())
# date3_feature['date3_dura_sum_percnt'] = date3_feature['date3_visit_dura_sum'] / x.values 
# 
# #APP/网站访问天数
# gp = date3.groupby(['uid','wa_type'])['date']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','3']
# date3_feature['date3_type0_date_mean'] = x['0']
# date3_feature['date3_type3_date_mean'] = x['3']
# 
# gp = date3.groupby(['uid','wa_type'])['date']
# x = gp.apply(lambda x:x.count())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','3']
# date3_feature['date3_type0_date_cnt'] = x['0']
# date3_feature['date3_type3_date_cnt'] = x['3']
# 
# #APP/网站访问总次数
# 
# gp = date3.groupby(['uid','wa_type'])['visit_cnt']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','3']
# date3_feature['date3_type0_visitcnt_mean'] = x['0']
# date3_feature['date3_type3_visitcnt_mean'] = x['3']
# 
# #APP/网站上载量
# gp = date3.groupby(['uid','wa_type'])['up_flow']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','3']
# date3_feature['date3_type0_upflow_mean'] = x['0']
# date3_feature['date3_type3_upflow_mean'] = x['3']
# #APP/网站下载量
# gp = date3.groupby(['uid','wa_type'])['down_flow']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','3']
# date3_feature['date3_type0_downflow_mean'] = x['0']
# date3_feature['date3_type3_downflow_mean'] = x['3']
# gp = date3.groupby(['uid','wa_type'])['flow']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','3']
# date3_feature['date3_type0_flow_mean'] = x['0']
# date3_feature['date3_type3_flow_mean'] = x['3']
# #APP/网站访问平均时长
# gp = date3.groupby(['uid','wa_type'])['visit_dura']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','3']
# date3_feature['date3_type0_dura_mean'] = x['0']
# date3_feature['date3_type3_dura_mean'] = x['3']
# 
# gp=date3.groupby('uid')['wa_name']
# x=gp.apply(lambda x:len(set(x)))
# date3_feature['date3_name_count_uinque']=x.values
# #APP/网站分别访问了多少种
# gp = date3.groupby(['uid','wa_type'])['wa_name']
# x = gp.apply(lambda x:len(set(x)))
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','3']
# date3_feature['date3_type0_name_unique'] = x['0']
# date3_feature['date3_type3_name_unique'] = x['3']

In [1243]:
# train_feature = train_feature.merge(date3_feature,on='uid',how='left').reset_index(drop=True)
# train_feature = train_feature.fillna(0)
# test_feature = test_feature.merge(date3_feature,on='uid',how='left').reset_index(drop=True)
# test_feature = test_feature.fillna(0)

In [1244]:
# date4 = pd.DataFrame()
# date4 = wa[wa['date']>27]
# date4 = date4[date4['date']<=36]

In [1245]:
# date4_feature = pd.DataFrame()
# #上网总天数
# gp = date4.groupby('uid')['date']
# x = gp.apply(lambda x:len(set(x)))
# date4_feature['uid'] = x.index
# date4_feature['date4_visit_date_cnt'] = x.values
# #访问次数
# # gp = date4.groupby('uid')['visit_cnt']
# # x = gp.apply(lambda x:x.sum())
# # date4_feature['date4_visit_cnt_sum'] = x.values
# # 
# # gp = date4.groupby('uid')['visit_cnt']
# # x = gp.apply(lambda x:x.max())
# # date4_feature['date4_visit_cnt_max'] = x.values
# # 
# # gp = date4.groupby('uid')['visit_cnt']
# # x = gp.apply(lambda x:x.std())
# # date4_feature['date4_visit_cnt_std'] = x.values
# # 
# # gp = date4.groupby('uid')['visit_cnt']
# # x = gp.apply(lambda x:x.median())
# # date4_feature['date4_visit_cnt_median'] = x.values
# # 
# gp = date4.groupby('uid')['visit_cnt']
# x = gp.apply(lambda x:x.mean())
# date4_feature['date4_visit_cnt_mean'] = x.values
# #访问时长
# # 
# gp = date4.groupby('uid')['visit_dura']
# x = gp.apply(lambda x:x.sum())
# date4_feature['date4_visit_dura_sum'] = x.values
# 
# # gp = date4.groupby('uid')['visit_dura']
# # x = gp.apply(lambda x:x.max())
# # date4_feature['date4_visit_dura_max'] = x.values
# # 
# # gp = date4.groupby('uid')['visit_dura']
# # x = gp.apply(lambda x:x.std())
# # date4_feature['date4_visit_dura_std'] = x.values
# # 
# # gp = date4.groupby('uid')['visit_dura']
# # x = gp.apply(lambda x:x.median())
# # date4_feature['date4_visit_dura_median'] = x.values
# 
# gp = date4.groupby('uid')['visit_dura']
# x = gp.apply(lambda x:x.mean())
# date4_feature['date4_visit_dura_mean'] = x.values
# 
# gp = date4.groupby('uid')['down_flow']
# x = gp.apply(lambda x:x.mean())
# date4_feature['date4_down_flow_mean'] = x.values
# 
# # gp = date4.groupby('uid')['down_flow']
# # x = gp.apply(lambda x:x.sum())
# # date4_feature['date4_down_flow_sum'] = x.values
# # 
# # gp = date4.groupby('uid')['down_flow']
# # x = gp.apply(lambda x:x.max())
# # date4_feature['date4_down_flow_max'] = x.values
# # 
# # gp = date4.groupby('uid')['down_flow']
# # x = gp.apply(lambda x:x.median())
# # date4_feature['date4_down_flow_median'] = x.values
# # 
# # gp = date4.groupby('uid')['down_flow']
# # x = gp.apply(lambda x:x.std())
# # date4_feature['date4_down_flow_std'] = x.values
# 
# gp = date4.groupby('uid')['up_flow']
# x = gp.apply(lambda x:x.mean())
# date4_feature['date4_up_flow_mean'] = x.values
# 
# # gp = date4.groupby('uid')['up_flow']
# # x = gp.apply(lambda x:x.sum())
# # date4_feature['date4_up_flow_sum'] = x.values
# # 
# # gp = date4.groupby('uid')['up_flow']
# # x = gp.apply(lambda x:x.max())
# # date4_feature['date4_up_flow_max'] = x.values
# # 
# # gp = date4.groupby('uid')['up_flow']
# # x = gp.apply(lambda x:x.std())
# # date4_feature['date4_up_flow_std'] = x.values
# # 
# # gp = date4.groupby('uid')['up_flow']
# # x = gp.apply(lambda x:x.median())
# # date4_feature['date4_up_flow_median'] = x.values
# 
# #总流量
# date4['flow'] = date4['down_flow'] + date4['up_flow']
# gp = date4.groupby('uid')['flow']
# x = gp.apply(lambda x:x.sum())
# date4_feature['date4_flow_sum'] = x.values
# 
# gp = date4.groupby('uid')['date']
# y = gp.apply(lambda x:len(set(x)))
# date4_feature['date4_flow_perday_mean'] = date4_feature['date4_flow_sum'] / y.values
# 
# gp = date4.groupby('uid')['visit_cnt']
# x = gp.apply(lambda x:x.sum())
# date4_feature['date4_flow_sum_percnt'] = date4_feature['date4_flow_sum'] / x.values 
# 
# gp = date4.groupby('uid')['visit_cnt']
# x = gp.apply(lambda x:x.sum())
# date4_feature['date4_dura_sum_percnt'] = date4_feature['date4_visit_dura_sum'] / x.values 
# 
# #APP/网站访问天数
# gp = date4.groupby(['uid','wa_type'])['date']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','4']
# date4_feature['date4_type0_date_mean'] = x['0']
# date4_feature['date4_type4_date_mean'] = x['4']
# 
# gp = date4.groupby(['uid','wa_type'])['date']
# x = gp.apply(lambda x:x.count())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','4']
# date4_feature['date4_type0_date_cnt'] = x['0']
# date4_feature['date4_type4_date_cnt'] = x['4']
# 
# #APP/网站访问总次数
# 
# gp = date4.groupby(['uid','wa_type'])['visit_cnt']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','4']
# date4_feature['date4_type0_visitcnt_mean'] = x['0']
# date4_feature['date4_type4_visitcnt_mean'] = x['4']
# 
# #APP/网站上载量
# gp = date4.groupby(['uid','wa_type'])['up_flow']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','4']
# date4_feature['date4_type0_upflow_mean'] = x['0']
# date4_feature['date4_type4_upflow_mean'] = x['4']
# #APP/网站下载量
# gp = date4.groupby(['uid','wa_type'])['down_flow']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','4']
# date4_feature['date4_type0_downflow_mean'] = x['0']
# date4_feature['date4_type4_downflow_mean'] = x['4']
# gp = date4.groupby(['uid','wa_type'])['flow']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','4']
# date4_feature['date4_type0_flow_mean'] = x['0']
# date4_feature['date4_type4_flow_mean'] = x['4']
# #APP/网站访问平均时长
# gp = date4.groupby(['uid','wa_type'])['visit_dura']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','4']
# date4_feature['date4_type0_dura_mean'] = x['0']
# date4_feature['date4_type4_dura_mean'] = x['4']
# 
# gp=date4.groupby('uid')['wa_name']
# x=gp.apply(lambda x:len(set(x)))
# date4_feature['date4_name_count_uinque']=x.values
# #APP/网站分别访问了多少种
# gp = date4.groupby(['uid','wa_type'])['wa_name']
# x = gp.apply(lambda x:len(set(x)))
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','4']
# date4_feature['date4_type0_name_unique'] = x['0']
# date4_feature['date4_type4_name_unique'] = x['4']

In [1246]:
# train_feature = train_feature.merge(date4_feature,on='uid',how='left').reset_index(drop=True)
# train_feature = train_feature.fillna(0)
# test_feature = test_feature.merge(date4_feature,on='uid',how='left').reset_index(drop=True)
# test_feature = test_feature.fillna(0)

In [1247]:
# date5 = pd.DataFrame()
# date5 = wa[wa['date']>36]
# date5 = date5[date5['date']<=45]

In [1248]:
# date5_feature = pd.DataFrame()
# #上网总天数
# gp = date5.groupby('uid')['date']
# x = gp.apply(lambda x:len(set(x)))
# date5_feature['uid'] = x.index
# date5_feature['date5_visit_date_cnt'] = x.values
# #访问次数
# # gp = date5.groupby('uid')['visit_cnt']
# # x = gp.apply(lambda x:x.sum())
# # date5_feature['date5_visit_cnt_sum'] = x.values
# # 
# # gp = date5.groupby('uid')['visit_cnt']
# # x = gp.apply(lambda x:x.max())
# # date5_feature['date5_visit_cnt_max'] = x.values
# # 
# # gp = date5.groupby('uid')['visit_cnt']
# # x = gp.apply(lambda x:x.std())
# # date5_feature['date5_visit_cnt_std'] = x.values
# # 
# # gp = date5.groupby('uid')['visit_cnt']
# # x = gp.apply(lambda x:x.median())
# # date5_feature['date5_visit_cnt_median'] = x.values
# # 
# gp = date5.groupby('uid')['visit_cnt']
# x = gp.apply(lambda x:x.mean())
# date5_feature['date5_visit_cnt_mean'] = x.values
# #访问时长
# # 
# gp = date5.groupby('uid')['visit_dura']
# x = gp.apply(lambda x:x.sum())
# date5_feature['date5_visit_dura_sum'] = x.values
# # 
# # gp = date5.groupby('uid')['visit_dura']
# # x = gp.apply(lambda x:x.max())
# # date5_feature['date5_visit_dura_max'] = x.values
# # 
# # gp = date5.groupby('uid')['visit_dura']
# # x = gp.apply(lambda x:x.std())
# # date5_feature['date5_visit_dura_std'] = x.values
# # 
# # gp = date5.groupby('uid')['visit_dura']
# # x = gp.apply(lambda x:x.median())
# # date5_feature['date5_visit_dura_median'] = x.values
# 
# gp = date5.groupby('uid')['visit_dura']
# x = gp.apply(lambda x:x.mean())
# date5_feature['date5_visit_dura_mean'] = x.values
# 
# gp = date5.groupby('uid')['down_flow']
# x = gp.apply(lambda x:x.mean())
# date5_feature['date5_down_flow_mean'] = x.values
# 
# # gp = date5.groupby('uid')['down_flow']
# # x = gp.apply(lambda x:x.sum())
# # date5_feature['date5_down_flow_sum'] = x.values
# # 
# # gp = date5.groupby('uid')['down_flow']
# # x = gp.apply(lambda x:x.max())
# # date5_feature['date5_down_flow_max'] = x.values
# # 
# # gp = date5.groupby('uid')['down_flow']
# # x = gp.apply(lambda x:x.median())
# # date5_feature['date5_down_flow_median'] = x.values
# # 
# # gp = date5.groupby('uid')['down_flow']
# # x = gp.apply(lambda x:x.std())
# # date5_feature['date5_down_flow_std'] = x.values
# 
# gp = date5.groupby('uid')['up_flow']
# x = gp.apply(lambda x:x.mean())
# date5_feature['date5_up_flow_mean'] = x.values
# 
# # gp = date5.groupby('uid')['up_flow']
# # x = gp.apply(lambda x:x.sum())
# # date5_feature['date5_up_flow_sum'] = x.values
# # 
# # gp = date5.groupby('uid')['up_flow']
# # x = gp.apply(lambda x:x.max())
# # date5_feature['date5_up_flow_max'] = x.values
# # 
# # gp = date5.groupby('uid')['up_flow']
# # x = gp.apply(lambda x:x.std())
# # date5_feature['date5_up_flow_std'] = x.values
# # 
# # gp = date5.groupby('uid')['up_flow']
# # x = gp.apply(lambda x:x.median())
# # date5_feature['date5_up_flow_median'] = x.values
# 
# #总流量
# date5['flow'] = date5['down_flow'] + date5['up_flow']
# gp = date5.groupby('uid')['flow']
# x = gp.apply(lambda x:x.sum())
# date5_feature['date5_flow_sum'] = x.values
# 
# gp = date5.groupby('uid')['date']
# y = gp.apply(lambda x:len(set(x)))
# date5_feature['date5_flow_perday_mean'] = date5_feature['date5_flow_sum'] / y.values
# 
# gp = date5.groupby('uid')['visit_cnt']
# x = gp.apply(lambda x:x.sum())
# date5_feature['date5_flow_sum_percnt'] = date5_feature['date5_flow_sum'] / x.values 
# 
# gp = date5.groupby('uid')['visit_cnt']
# x = gp.apply(lambda x:x.sum())
# date5_feature['date5_dura_sum_percnt'] = date5_feature['date5_visit_dura_sum'] / x.values 
# 
# #APP/网站访问天数
# gp = date5.groupby(['uid','wa_type'])['date']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','5']
# date5_feature['date5_type0_date_mean'] = x['0']
# date5_feature['date5_type5_date_mean'] = x['5']
# 
# gp = date5.groupby(['uid','wa_type'])['date']
# x = gp.apply(lambda x:x.count())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','5']
# date5_feature['date5_type0_date_cnt'] = x['0']
# date5_feature['date5_type5_date_cnt'] = x['5']
# 
# #APP/网站访问总次数
# 
# gp = date5.groupby(['uid','wa_type'])['visit_cnt']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','5']
# date5_feature['date5_type0_visitcnt_mean'] = x['0']
# date5_feature['date5_type5_visitcnt_mean'] = x['5']
# 
# #APP/网站上载量
# gp = date5.groupby(['uid','wa_type'])['up_flow']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','5']
# date5_feature['date5_type0_upflow_mean'] = x['0']
# date5_feature['date5_type5_upflow_mean'] = x['5']
# #APP/网站下载量
# gp = date5.groupby(['uid','wa_type'])['down_flow']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','5']
# date5_feature['date5_type0_downflow_mean'] = x['0']
# date5_feature['date5_type5_downflow_mean'] = x['5']
# gp = date5.groupby(['uid','wa_type'])['flow']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','5']
# date5_feature['date5_type0_flow_mean'] = x['0']
# date5_feature['date5_type5_flow_mean'] = x['5']
# #APP/网站访问平均时长
# gp = date5.groupby(['uid','wa_type'])['visit_dura']
# x = gp.apply(lambda x:x.mean())
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','5']
# date5_feature['date5_type0_dura_mean'] = x['0']
# date5_feature['date5_type5_dura_mean'] = x['5']
# 
# gp=date5.groupby('uid')['wa_name']
# x=gp.apply(lambda x:len(set(x)))
# date5_feature['date5_name_count_uinque']=x.values
# #APP/网站分别访问了多少种
# gp = date5.groupby(['uid','wa_type'])['wa_name']
# x = gp.apply(lambda x:len(set(x)))
# x = x.unstack(fill_value=0).reset_index(drop=True)
# x.columns = ['0','5']
# date5_feature['date5_type0_name_unique'] = x['0']
# date5_feature['date5_type5_name_unique'] = x['5']

In [1249]:
# train_feature = train_feature.merge(date5_feature,on='uid',how='left').reset_index(drop=True)
# train_feature = train_feature.fillna(0)
# test_feature = test_feature.merge(date5_feature,on='uid',how='left').reset_index(drop=True)
# test_feature = test_feature.fillna(0)

In [1250]:
train_feature.to_csv('train_feature_v1.csv',index=None)
test_feature.to_csv('test_feature_b_v1.csv',index=None)

In [1251]:
train_feature.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4999 entries, 0 to 4998
Columns: 229 entries, uid to wa_type1_down_perday
dtypes: float64(225), int64(3), object(1)
memory usage: 8.7+ MB
